#Imports

In [ ]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install('geopandas')
install('geemap')
install('haversine') # Não sei se é assim que deve ser feito
install('rtree')
install('pygeos')
install('rasterio')

import geopandas as gpd
import rasterio
import rtree, pygeos, shapely
import ee, os, geemap, glob
import pandas as pd # lembrar de trocar para o pandas 1.3 (o mais novo)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import gc
import haversine as hs # Novo
from haversine.haversine import Unit # Novo
import datetime
import time
import random
from scipy.spatial import cKDTree
from shapely.geometry import Point

ee.Authenticate() # quando este passo nao foi feito obtive erro no geemap.shp_to_ee
ee.Initialize()

#Funcoes base

####Adicionar índices e filtrar imagens Sentinel 2

In [ ]:
#compute ndvi
def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).multiply(10000).int16()
  return image.addBands(ndvi.rename('ndvi'))

#compute ndwi
def addNDWI(image):
  ndwi = image.normalizedDifference(['B3', 'B8']).multiply(10000).int16()
  return image.addBands(ndwi.rename('ndwi'))

#compute EVI
def addEVI(image):
  evi = image.expression(
  '2.5 * ((NIR-RED) / (NIR + 6 * RED - 7.5* BLUE +1))', {
    'NIR':image.select('B8').divide(10000),
    'RED':image.select('B4').divide(10000),
    'BLUE':image.select('B2').divide(10000)    
  }).multiply(10000).int16()                                                        
  return image.addBands(evi.rename('evi'))

#compute NBR
def addNBR(image):
  nbr = image.normalizedDifference(['B8', 'B12']).multiply(10000).int16()
  return image.addBands(nbr.rename('nbr'))

#compute NBR2
def addNBR2(image):
  nbr = image.normalizedDifference(['B11', 'B12']).multiply(10000).int16()
  return image.addBands(nbr.rename('nbr2'))

#SCL cloud/shadow filter
def filterS2_level2A(image):
  SCL = image.select('SCL')
  mask01 = ee.Image(0).where((SCL.lt(8)).And(SCL.gt(3)),1)   #Put a 1 on good pixels
 #(SCL.gt(3),1)
  return image.updateMask(mask01)

#s2cloudless filter
def filterS2cloudless(S2SRCol, S2CloudCol):

  CLOUD_FILTER = 60;
  CLD_PRB_THRESH = 50;
  NIR_DRK_THRESH = 0.2;
  CLD_PRJ_DIST = 1;
  BUFFER = 50;

  #filter images based on cloudy percentage (metadata)
  S2SRCol = S2SRCol.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))

  #join S2SR with S2CloudCol
  joined = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(
      primary = S2SRCol,
      secondary = S2CloudCol,
      condition = ee.Filter.equals(
          leftField = 'system:index',
          rightField = 'system:index'
      )))
  
  def add_cloud_bands(img):
    #Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    #Condition s2cloudless by the probability threshold value
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    #Add the cloud probability layer and cloud mask as image bands
    return img.addBands(ee.Image([cld_prb, is_cloud]))
  
  def add_shadow_bands(img):
    #Identify water pixels from the SCL band
    not_water = img.select('SCL').neq(6)
    #Identify dark NIR pixels that are not water (potential cloud shadow pixels)
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    #Determine the direction to project cloud shadow from clouds (assumes UTM projection)
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    #Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(crs=img.select(0).projection(), scale=100)
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    #Identify the intersection of dark pixels with cloud shadow projection
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    #Add dark pixels, cloud projection, and identified shadows as image bands
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

  def add_cld_shdw_mask(img):
    #Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    #Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)
    #Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    #Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input
    #20 m scale is for speed, and assumes clouds don't require 10 m precision
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(crs=img.select([0]).projection(), scale= 20)
        .rename('cloudmask'))
    #Add the final cloud-shadow mask to the image
    return img_cloud_shadow.addBands(is_cld_shdw)

  def apply_cld_shdw_mask(img):
    #Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not();
    #Subset reflectance bands and update their masks, return the result.
    return img.updateMask(not_cld_shdw)

  s2_sr = joined.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

  return s2_sr



####getImageCollection - Carregar coleção de imagens Sentinel-2

In [ ]:
def getImageCollection(params_ImgCol, sites, buf=False, clip=True):
  
  """Function that takes the collection name, start and end dates and
  a geometry that is going to be used to clip the image.
  Returns an image collection for the bounding box that contains the points defined in sites 
  (and buffer if that is the case)
  Arguments: 
  params_ImgCol - dictionary containing:
    date_start/date_end - dates used as temporal filter;
    indices - indices that must be added to the images (only supports NDVI, NDWI and EVI);
  sites - geometry used to get the spatial location and also clip the images;
  clip - whether to clip the image collection to the extent of the sites geometry;
  buf - buffer size used in case of analysis of area surrounding a point."""

  #define spatial extent
  if buf: # valor do buffer para o caso de procurar a informação do entorno
    export_extent = sites.geometry().buffer(buf).bounds()
  else:
    #check if sites is a feature collection. If not (meaning it's a single point), the method bounds should not be used
    if ee.Algorithms.ObjectType(sites).getInfo() == 'FeatureCollection':
      export_extent = sites.geometry().bounds()
    else:
      export_extent = sites.geometry()

  #get collection by its name and filter with the spatial extent of the sites and dates
  S2 = ee.ImageCollection(params_ImgCol['nameImage']).filterBounds(export_extent).filterDate(params_ImgCol['date_start'],params_ImgCol['date_end'])
  
  #filter clouds
  if params_ImgCol['cloudFilter'] == 'SCL': #apply cloud filter based on SCL values
    S2filtered= S2.map(filterS2_level2A)

  elif params_ImgCol['cloudFilter'] == 's2cloudless': #apply cloud filter based on the s2cloudless procedure
    #load cloud probability collection
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(export_extent)
                                                                            .filterDate(params_ImgCol['date_start'],params_ImgCol['date_end']))
    S2filtered = filterS2cloudless(S2, s2_cloudless_col)

  #check and add spectral indices entered by user (NDVI, NDWI, EVI, NBR) 
  if 'ndvi' in list(map(lambda x: x.lower(), params_ImgCol['indices'])):
    S2filtered = S2filtered.map(addNDVI)
  if 'ndwi' in list(map(lambda x: x.lower(), params_ImgCol['indices'])):
    S2filtered = S2filtered.map(addNDWI)
  if 'evi' in list(map(lambda x: x.lower(), params_ImgCol['indices'])):
    S2filtered = S2filtered.map(addEVI)
  if 'nbr' in list(map(lambda x: x.lower(), params_ImgCol['indices'])):
    S2filtered = S2filtered.map(addNBR)
  if 'nbr2' in list(map(lambda x: x.lower(), params_ImgCol['indices'])):
    S2filtered = S2filtered.map(addNBR2)

  for i in params_ImgCol['indices']:
    if i.lower() not in ['ndvi','ndwi','evi','nbr','nbr2']:
      print('Adding indice {} is currently not supported on our code. The returned collection will lack such indice'.format(i))

  #function to clip image to selected region.
  def Clip(image):
    clipImage = image.clip(export_extent)
    return clipImage

  #clip to selected region
  if clip:
    S2filtered_clipped = S2filtered.map(Clip) #colecao de imagens utilizadas
  
  return S2filtered_clipped if clip else S2filtered 
  

####runCCDC

In [ ]:
def runCCDC(collection, bandas, params_ccdc):
  """
  Entrada:
   - collection: objeto Image Collection
   - bandas: lista com as bandas selecionadas para a analise
   - params_ccdc: dicionario com os parametros do ccdc
  Saida:
   - ccdc_result: Image Array do resultado do CCDC  
  """
  
  # B2-Blue, B3-Green, B4-Red, B8-NIR, B12-SWIR2
  ccdc_result = ee.Algorithms.TemporalSegmentation.Ccdc(collection.select(bandas), #colecao
                                                params_ccdc['bandas_breakpoint'], #breakpointBands                                               
                                                params_ccdc['bandas_tmask'], #tmaskBands 
                                                params_ccdc['minObs'], #minObservations
                                                params_ccdc['chiSquare'], #chiSquareProbability
                                                params_ccdc['minYears'], #minNumOfYearsScaler
                                                params_ccdc['dateForm'], #dateFormat 
                                                params_ccdc['Lambda'], #lambda para NDVI normalizado * 10000
                                                params_ccdc['maxIter']) #maxIterations
  return ccdc_result

#Função filtro data frame CCDC; não usa dados dos analistas (apenas as datas)

In [ ]:
def filterDate(pathDF, dataI, dataF,bandFilter, mag = None):
    """
    Reduz o número de linhas do data frame de entrada, removendo as linhas fora do período de análise e
    para o limite estabelecido de magnitude máxima.
    Entrada:
        pathDF: caminho do Data Frame do CCDC
        dataI: String com a data inicial na forma = 'AAAA-MM-DD' (e.g. a data inicial dos analistas nos pontos DGT 300)
        dataF: String com a data final na forma = 'AAAA-MM-DD' (e.g. a data final dos analistas nos pontos DGT 300)
        bandFilter: String com a banda para a qual se deseja filtrar os dados. A esta banda é aplicado o criterio do mag.
        mag: Número com o limite da magnitude, e.g 0 só serão utilizadas as linhas com magnitudo menor ou igual a zero
    Saída:
        Data Frame filtrado
    """
    # Data Frame CCDC
    if pathDF.endswith('.csv'):
        df = pd.read_csv(pathDF, delimiter = ';')
    if pathDF.endswith('.pkl'):
        df = pd.read_pickle(pathDF)
        
    for dtCol in df.columns:
        if 'tBreak' in dtCol or 'tEnd' in dtCol or 'tStart' in dtCol:
            mask = df.loc[:, dtCol] == 0
            df[dtCol] = pd.to_datetime(df[dtCol], unit = 'ms')
            df.loc[mask, dtCol] = np.nan
        elif 'End_S' in dtCol:
            df[dtCol] = pd.to_datetime(df[dtCol]) # Esta coluna inicialmente esta em formato texto
    df.rename(columns={ 'Unnamed: 0':'IDCCDC'}, inplace=True)  
    
    if mag != None:
        # caso haja magnitude limite, colocar tudo como NAT que seja acima deste limite
        df.loc[df[bandFilter] > mag, 'tBreak'] = pd.to_datetime(np.nan)
        df = df.copy()
    else:
        df = df.copy()
        
    # filtro das datas
    yi, mi, diai = convertDate(dataI)
    fltInicial = datetime.datetime(yi, mi, diai)
    yf, mf, diaf = convertDate(dataF)
    fltFinal = datetime.datetime(yf, mf, diaf)           
    
    # 1 Adiciona a coluna com a menor data de start do fit
    df['startMin'] = df.groupby(['coord_ccdc'])['tStart'].transform('min')

    # 2 Adiciona o número de breaks existentes num grupo de IDCCDC, independente de fltInicial e fltFinal
    df['numBreak'] = np.ceil(df.groupby(['coord_ccdc'])['changeProb'].transform('sum'))
    
    # Colocar Nat nas probabilidades fracionadas
    df.loc[((df.changeProb > 0) & (df.changeProb < 1)), 'tBreak'] = pd.to_datetime(np.nan)

    # 3 Verifica se se os breaks estão dentro do período de análise e transforma em NaT todos os que não estão
    df['breaks_in_tmask'] = (~df.tBreak.isnull()).astype(int)
    df.loc[(df['tBreak'] <= fltInicial) | (df['tBreak'] >= fltFinal), 'breaks_in_tmask'] = 0
    df.loc[(df['tBreak'] <= fltInicial) | (df['tBreak'] >= fltFinal), 'tBreak'] = np.nan

    # Mascaras necessárias
    # a) Verifica os breaks NaT para as linhas com mais de 1 break
    mask = pd.Series(np.zeros(len(df),dtype=bool),index = df.index)
    mask.loc[(df.tBreak.isnull()) & (df.numBreak > 1)]= True #cond3
    
    # b) Verifica nas linhas de 1 break e sejam nulos qual é aquele que tem o início da série,
    #pois caso esteja fora da data de análise deve ser eliminado
    nmask = pd.Series(np.zeros(len(df),dtype=bool),index = df.index)
    nmask.loc[(df.tBreak.isnull()) & (df.numBreak == 1) & (df.breaks_in_tmask == 0) & (df.tStart == df.startMin)]= True    
    
    # Aplica as mascaras acima e gera um novo DF
    subset_Filtro = df[((mask == False) & (nmask == False))].copy()
    
    # c) Calcula quantos linhas há por IDCCDC e caso ainda existam 2 significa que o break está dentro do período de análise e o fit final, sem break
    # deve ser eliminado
    smask = pd.Series(np.zeros(len(subset_Filtro),dtype=bool),index = subset_Filtro.index)
    smask.loc[(subset_Filtro.groupby(['coord_ccdc'])['IDCCDC'].transform('count') == 2) & (subset_Filtro.changeProb == 0) & (df.numBreak == 1)] = True
    subset_Filtro = subset_Filtro[(smask == False)].copy()
    
    # d) Para os IDCCDC que apresentam linhas com probabilidade fracionada, mantem esta linha, no caso de todas estarem fora do período de análise
    pmask = pd.Series(np.zeros(len(df),dtype=bool),index = df.index)
    pmask.loc[~((df.changeProb > 0) & (df.changeProb < 1) & (df.tBreak.isnull()) & (df.groupby(['coord_ccdc'])['tBreak'].transform('count') == 0))]=True
    subset_Filtro = subset_Filtro.append(df[pmask == False])
    
    # e) Para os IDCCDC que tem mais de um break e todos estao fora do periodo e devemos manter o fit final
    fmask = pd.Series(np.zeros(len(df),dtype=bool),index = df.index)
    fmask.loc[((df.changeProb == 0) & (df.numBreak > 1) & (df.tBreak.isnull()) & (df.groupby(['coord_ccdc'])['tBreak'].transform('count') == 0))]=True
    subset_Filtro = subset_Filtro.append(df[fmask])
    

    return subset_Filtro

#Funcoes Gerar Graficos

####ccdcTimeseries - Gerar e selecionar os segmentos de reta; by parevalo

In [ ]:
def ccdcTimeseries(collection, proj, dateFormat, ccdc, geometry, band, padding):

  """Calculate and returns a time series of CCDC values for the dates with observations based on the harmonic coefficients"""

  def harmonicFit(t, coef):
    #returns the result of the harmonic regression for a given time t, given the coefficients' values
    PI2 = 2.0 * np.pi
    OMEGAS = [PI2 / 365.25, PI2, PI2 / (1000 * 60 * 60 * 24 * 365.25)]
    omega = OMEGAS[dateFormat]
    return coef.get([0]).add(coef.get([1]).multiply(t)).add(coef.get([2]).multiply(t.multiply(omega).cos())).add(coef.get([3]).multiply(t.multiply(omega).sin())).add(coef.get([4]).multiply(t.multiply(omega * 2).cos())).add(coef.get([5]).multiply(t.multiply(omega * 2).sin())).add(coef.get([6]).multiply(t.multiply(omega * 3).cos())).add(coef.get([7]).multiply(t.multiply(omega * 3).sin()))
  
  def convertDateFormat(date, format):
    #converts date to the intended format
    if format == 0:
      epoch = 719529
      days = date.difference(ee.Date('1970-01-01'), 'day')
      return days.add(epoch)
    elif format == 1:
      year = date.get('year')
      fYear = date.difference(ee.Date.fromYMD(year, 1, 1), 'year')
      return year.add(fYear)
    else:
      return date.millis()

  def date_to_segment(t, fit):
    tStart = ee.Array(fit.get('tStart'))
    tEnd = ee.Array(fit.get('tEnd'))
    return tStart.lte(t)and(tEnd.gte(t)).toList().indexOf(1)

  def produceTimeSeries(collection, ccdc, geometry, band):
    ccdcFits = ccdc.reduceRegion(reducer=ee.Reducer.first(), geometry=geometry, crs= proj)
    if padding:
      collection = collection.sort('system:time_start')
      first = collection.first()
      last = collection.sort('system:time_start', False).first()
      def aux1(t):
        fYear = ee.Number(t)
        year = fYear.floor()
        return  ee.Date.fromYMD(year, 1, 1).advance(fYear.subtract(year), 'year')

      fakeDates = ee.List.sequence(first.date().get('year'), last.date().get('year'), padding).map(aux1)
      def aux2(d):
        return ee.Image().rename(band).set('system:time_start', ee.Date(d).millis())

      fakeDates = fakeDates.map(aux2)
      collection = collection.merge(fakeDates)
       
    collection = collection.sort('system:time_start')
  
    #Augment images with the model fit
    def aux3(img):
      time = convertDateFormat(img.date(), dateFormat)
      segment = date_to_segment(time, ccdcFits)
      value = img.select(band).reduceRegion(reducer=ee.Reducer.first(), geometry=geometry, crs=proj).getNumber(band)
      coef = ee.Algorithms.If(segment.add(1), ccdcFits.getArray(band + '_coefs').slice(0, segment, segment.add(1)).project([1]),ee.Array([0,0,0,0,0,0,0,0,0]))
      fit = harmonicFit(time, ee.Array(coef))

      #return img.set(value=value, fitTime=time, fit=fit, coef=coef, segment=segment, dateString=img.date().format("YYYY-MM-dd")).set(segment.format("h%d"), fit)
      return img.set({'value':value, 'fitTime':time, 'fit':fit, 'coef':coef, 'segment':segment, 'dateString':img.date().format("YYYY-MM-dd")}).set(segment.format("h%d"), fit)
    
    timeSeries = collection.map(aux3)      
    return timeSeries  
  return produceTimeSeries(collection, ccdc, geometry, band)

####chartCcdc - Dados para o chart

In [ ]:
def chartCcdc(collection, ccdc_result, runParams, geometry, band): # aqui entraria a banda
  
  """
  Returns a dataframe containing a time series with values from actual observations and the respective CCDC values.
  Also provides information about each segment's coefficients.
  Dataframe structure: date - observation date; 
  obs - observation value; 
  a, b, c, d, e, f - CCDC value for fits 1 (a), 2 (b), 3 (c)...
  g - CCDC value regardless of fit.
  
  Arguments: collection - ee Image collection;
  ccdc_result - output from ccdc ee algorithm;
  runParams - dictionary containing run info. In this case it needs the key nSegs, that indicates the desired number of segments;
  geometry - the geometry (ee.geometry.Geometry) of the point where one wants to obtain the CCDC chart data;
  band - the band of interest;
  """

  # Set up and run CCDC
  # Need to filter bands because indices code does not currently work if TEMP is included
  #collection = S2filtered_clipped.select(bandas)
  #collection = collection.select(['ndvi', 'B2', 'B3','B4','B8', 'B11', 'B12'])
    
  # Snap click box to image
  ref_image =ee.Image(collection.select(band).first()) 
  proj = ref_image.projection().atScale(10)
  
  series = ccdcTimeseries(collection, proj, 2, ccdc_result, geometry, band, 0.1) # alterar a banda

  def aux4(p):
    return ee.Number(p).floor()
  c1 = geometry.transform(proj, 1).coordinates().map(aux4)
  def aux5(p):
    return ee.Number(p).add(1)
  c2 = c1.map(aux5)
  p2 =  ee.Geometry.LineString([c1, c2], proj)
  
  # Get required list programatically for n segments
  templist = ["dateString", "value" ]
  for i in range(0, runParams['nSegs']): 
    templist.append("h"+str(i))

  templist.append("fit")
  listLength = len(templist)
  
  table = series.reduceColumns(ee.Reducer.toList(listLength, listLength), templist).get('list')
  #print(table.getInfo())
  df = pd.DataFrame(table.getInfo())
  df.columns= ['date','obs','a','b','c','d','e','f','g']
  return df

####runChartccdc - Run chart CCDC

In [ ]:
def runChartccdc(collection, ccdc_result, runParams, point, band, validation=None, ponto_qualquer=False):

  """
  Returns a dictionary whose key is the point and whose value is another dictionary containing the dataframe with time series (value)
  of a given band (key).
  Arguments: collection - ee Image collection;
  runParams - dictionary containing run info. In this case it needs the key nSegs, that indicates the desired number of segments;
  point -  OBID number (for ponto_qualquer=False) or an ee.Feature from a specific chosen point (ponto_qualquer=True);
  band -  band on which the analysis is focused;
  validation - ee feature collection with validation points from DGT.
  """

  #check if the execution is targeted to the validation points or for a specific chosen point
  if ponto_qualquer == False: ##DELETAR APOS TESTES???
    #check if validation is a feature collection
    if ee.Algorithms.ObjectType(validation).getInfo() == 'FeatureCollection':
      #select feature from validation points based on the OBID
      feature = validation.filter(ee.Filter.eq('OBID', point))
    else: #assumes validation is an ee.feature (single point entered manually)
      feature = validation
  else:
    feature = point
    #if it's a point outside the validation dataset, variable point gets assigned to -1 to ensure compatibility with the rest of the code that was written for the validation dataset
    point = (feature.get('OBID')).getInfo()
        
  return {point:{band: chartCcdc(collection, ccdc_result, runParams, feature.geometry(), band)}}

####takeInfo - Informacao dos pontos de validacao para nomear o grafico

In [ ]:
def takeInfo(gpdOb, num):

  """
  Returns information regarding the change dates of each analyst and
  land cover type as given by COS. It is used to set the chart's filename and other
  chart elements.
  Arguments: 
  gpdOb - geopandas dataframe corresponding to the validation shapefile;
  num - OBID number from which one wants to obtain info.
  """

  # dicionario para guardar os valores e possiveis combinacoes da tabela
  dic = {'bin':{}, 'tip':{}, 'dat':{}}  
  analistas = ['1_x','2_x','1_y','2_y','1_z','2_z']
  #filtrar o GDF
  filter = gpdOb.query('OBID == {0}'.format(num))  
  lat,log = (round(float(filter.geometry.y), 6), round(float(filter.geometry.x), 6))
  cobertura = filter.COS2018_Lg.values[0]
  # funcoes
  def binario(val):
    if val:
      valor = '1'
    else:
      valor = '0'
    return valor
  def alteracao(tipo):
    if tipo:
      alt = tipo
    else:
      alt = '-'
    return alt  
  for analista in analistas:       
    dic['bin'][analista] = binario(filter['tipo'+analista].values[0])
    dic['tip'][analista] = alteracao(filter['tipo'+analista].values[0])    
    dic['dat'][analista] = filter['data'+ analista].values[0]
  datas = dic['dat']
  alteracoes = dic['tip']
  arquivo = 'X{0}{1}_Y{2}{3}_Z{4}{5}'.format(dic['bin']['1_x'],dic['bin']['2_x'],dic['bin']['1_y'],
                                             dic['bin']['2_y'], dic['bin']['1_z'],dic['bin']['2_z'])
  return((cobertura,arquivo, datas,alteracoes, lat, log))

####generateChart - Gerar os graficos

In [ ]:
def generateChart(dfs, band, point, gpd_validation, date_end, ponto_qualquer=False, export=False, output_path=False):

    """
    Function used to generate chart of observations and ccdc regression fits.
    Arguments: 
    dfs - dictionary obtained from the runChartccdc function;
    band - the band in which the analysis in based on;
    point - either an OBID number or an ee.feature;
    gpd_validation - validation points shapefile read as a geodataframe;
    date_end - date of the end of the series;
    export - if set to True, function will export the chart figure to google drive; 
    if set to False, it will only exhibit the chart. Note that when running the code
    for all validation points, export should be True.
    output_path - directory where charts will be exported to.    
    """

    #set point value in order to access the correct key in the dfs dictionary
    if ponto_qualquer == True:
      long_ponto = ee.Number(point.geometry().coordinates().get(0)).getInfo()
      lat_ponto = ee.Number(point.geometry().coordinates().get(1)).getInfo()
      point = (point.get('OBID')).getInfo()

    #obid = list(df_elem.keys())[0]
    ts = dfs[point][band]
    #ts = dfs[obid]

    #convert date column to the correct format
    ts['date'] = pd.to_datetime(ts['date'])

    #get first date with observation
    first_date = ts[(ts['a'] > -10000) & (ts['obs'] > -10000)].iloc[0].date
    #delete rows with date prior to first date
    ts = ts[ts['date'] >= first_date]
 
    #remove empyt columns
    c = ts.columns
    for j in c:  
        if ts[j].count() == 0:
          ts = ts.drop(columns = [j])

    #generate chart
    #configure chart elements
    sns.set_style("darkgrid") #adds a grid to the chart
    leter = ['b', 'c', 'd','e', 'f'] #letras das colunas
    colors = ['green', 'purple', 'pink', 'yellow', 'orange'] # cores das linhas
    colunas = {*ts} - {'system:index', 'date', '.geo', 'g', 'a', 'obs'}
    plt.figure(figsize=(13, 6), dpi = 300)
    plt.scatter(ts['date'], ts['obs'], s=2.5, c = 'r', label= band.upper()) # era: 'NDVI'
    plt.plot(ts['date'],ts['a'], label='Fit1')
    for n, item in enumerate(colunas):
        if item in leter:
          Label = 'Fit{0}'.format(n+2)
          plt.plot(ts['date'],ts[str(leter[n])],c = colors[n], label='Fit{0}'.format(n+2))


    #This part runs only if the analysis is targeted to the validation points from DGT
    #if point != -1 :
    try:
      # get shapefile values and infos
      cosValue,analistasDGT, datasDGT,alteracoesDGT, latDGT, logDGT = takeInfo(gpd_validation, point)    

      # insert DGT date in plot
      ptColors = ['red','green', 'purple', 'pink', 'black', 'orange']
      ptTypes =['s', 's', 'd', 'd', 'p','p' ]
      for ptN, k in enumerate(datasDGT.keys()):  ###################################
        if datasDGT[k] != None:
          if int(datasDGT[k]) > 0:          
            d = pd.to_datetime(datasDGT[k],format='%Y%m%d', errors='ignore')
            plt.scatter(d, ts['obs'].min(), s=20, c = ptColors[ptN],marker= ptTypes[ptN], label= '{0}: {1}'.format(k, alteracoesDGT[k]))

      #numero de segmentos
      if colunas:
        segm = len(colunas) + 1
      else:
        segm = 1
      
      #set chart title
      plt.title('Cobertura COS: {0}. Lat: {1}, Long: {2}'.format(cosValue, latDGT, logDGT))
      #set chart filename
      chartName = 'OBID_{0}_{1}_{2}_{3}_Fits{4}_tMASK.png'.format(point, date_end.replace('-', ''), band.upper(), analistasDGT, segm)# OBS: date_end definido fora da funcao
    
    #configure axis labels and add legend
    #special rule for chart title and filename if the analysis is targeted to a specific user defined point
    #if point == -1:
      #plt.title('Análise de Ponto Específico (Lat: {0}, Long: {1})'.format(lat_ponto,long_ponto))
      #chartName = 'PontoEspecifico_{0}_{1}.png'.format(lat_ponto,long_ponto)
    except:
      plt.title('OBID: {}'.format(point))
      chartName = 'OBID_{0}_{1}_{2}_tMASK.png'.format(point, date_end.replace('-', ''), band.upper())
    #set other chart parameters that are applied regardless of the analysis target
    plt.xlabel('Data')
    plt.ylabel(band.upper()) # era 'NDVI'
    plt.legend()

    
    #display chart on notebook cell or save (export) chart to drive    
    if export == True:
      matplotlib.use('agg') #switch matplotlib backend from inline to agg (prevents memory leak)
      plt.savefig(output_path + chartName) # caso nao tenha subpasta alterar
      print('Chart exported: ' + output_path + chartName)
      plt.close() #prevent displaying the charts in the notebook cell output
      plt.clf() #clear figure
      gc.collect() #attempt to free memory
    else:
      print(chartName)
      plt.show()

####chartAllpoints - Gerar para todos os pontos

In [ ]:
from threading import Thread

#this is a Thread subclass that is modified so that it can return the function value when
#using threads
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return


def chartAllpoints(collection, ccdc_result, runParams, banda, ee_validation, gpd_validation, date_end, output_path):

  """Function used to generate charts for all points in the validation dataset.
  It runs the function runChartccdc for each OBID in blocks of 10. Then it stores the
  resulting dataframes in a dictionary (dfs). After that, it loops through all points
  in the dictionary and executes the generateChart function.
  Arguments: collection - ee Image collection;
  banda - band of analysis;
  date_end - date of the end of the series;
  ee_validation - validation dataset as ee feature collection;
  gpd_validation - validation dataset as a geodataframe;
  output_path - directory where the charts will be saved.
  """
  backup_stdout = sys.stdout #create backup of stdout
  sys.stdout = open(os.devnull,'w') #this aims to avoid printing out some unnecessary text during the function execution
  dfs = {}

  points_number = len(gpd_validation)
  #code used to run function runChartccdc in blocks of 10
  threads = []
  for i in range(0,points_number,5):
    threads = []
    for j in range(0,5):
      if i+j < points_number:
        t = ThreadWithReturnValue(target=runChartccdc, args=[collection, ccdc_result, runParams, i+j, banda, ee_validation])
        t.start()
        threads.append(t)
    for t in threads:
      thread_return = t.join()  
      dict_key = list(thread_return.keys())[0]
      dfs[dict_key] = thread_return[dict_key]
  
  #check if the execution crashed for some OBIDs (thread exception) and run again for the missing OBIDs
  missing = [x for x in dfs.keys() if x not in range(0,points_number)]
  while len(missing) > 0:
    for i in range(0,len(missing),5):
      threads = []
      for j in range(0,5):
        t = ThreadWithReturnValue(target=runChartccdc, args=[collection, ccdc_result, runParams, missing[i+j], banda, ee_validation])
        t.start()
        threads.append(t)
      for t in threads:
        thread_return = t.join()
        dict_key = list(thread_return.keys())[0]
        dfs[dict_key] = thread_return[dict_key]  
        
    missing = [x for x in dfs.keys() if x not in range(0,points_number)]
  
  sys.stdout = backup_stdout #restore stdout to normal, after being modified in the beginning of the function

  #after filling up the dictionary dfs, run function generateChart for each point
  for i in range(points_number):
    if points_number == 1:
      generateChart(dfs, banda, i, gpd_validation, date_end, export=False, output_path=output_path)
    #by default, generation of charts for all points is intended to export chart to drive, not to exhibit it in notebook
    else:
      generateChart(dfs, banda, i, gpd_validation, date_end, export=True, output_path=output_path)
      gc.collect() #attempt to free memory
  

#Funcoes Importar Info para Pontos

####getLatLong - ANTIGA

In [ ]:
# def getLatLong(area):

#   """Cria uma imagem contendo a lat long dos pixeis e recorta para a area determinada"""

#   lat_long = ee.Image.pixelLonLat().clip(area) #versão antiga dessa linha utilizava uma função especifica criada por nós para fazer o clip
  
#   latDic = lat_long.reduceRegion(
#     reducer= ee.Reducer.toList(),
#     geometry= area,
#     scale= 10)
#   return pd.DataFrame(latDic.getInfo()) # retorna a informacao das coordenadas em um dataframe

####Coletar info para os pontos - ccdcPoint, ccdcStudyArea & ccdcReducer

In [ ]:
# AS DUAS PROXIMAS FUNCOES SAO PARA COLETAR O VALOR DO CCDC EM UM PONTO OU EM
#ÁREA. OBS: TENTEI FAZER COM O BUFFER 0 PARA VER SE O RESULTADO ERA IGUAL AO 
# OBTIDO PARA SÓ UM PONTO, MAS NÃO DEU CERTO.

def ccdcPoint(pointGeom, ccdcResult):
  """ 
  Coletar as informacoes do CCDC para um ponto especifico
      input:
  pointGeom: Area do Ponto a reduzir o CCDC
  ccdcResult: Imagem resultante do algoritmo CCDC
      output:
  data frame contendo as informacoes no ponto selecionado
  """
  point_study = ccdcResult.reduceRegion(
    reducer = ee.Reducer.first(), #usar quando quer 3 linhas      
    geometry = pointGeom.geometry(),      
    scale= 10)
  return pd.DataFrame(point_study.getInfo())

def multipleCcdcPoint(fc, id_field, ccdc_result):

  dfs = {}
  #code used to run function ccdcPoint in blocks of 10
  for i in range(0,fc.size().getInfo(),10):
    threads = []
    threads_return = []
    for j in range(0,10):
      t = ThreadWithReturnValue(target=ccdcPoint, args=[fc.filter(ee.Filter.eq(id_field, i+j)), ccdc_result])
      t.start()
      threads.append(t)
    for t in threads:
      threads_return.append(t.join())
    for k in range(len(threads_return)):
      dfs[i + k] = threads_return[k]
    
  
  return dfs
      
  
def ccdcStudyArea(area, ccdcResult):
  """
  Coletar a informacao do CCDC para o entorno de um ponto especifico
    input:
    pointGeom: Geometria do ponto
    Buffer: Distancia do entorno do ponto para qual deseja-se a informacao
    ccdcResult: Resultado do ccdc
    output:
    dataframe contendo as informacoes sobre a area de estudada
  """
  ccdc_study_area = ccdcResult.reduceRegion(    
    reducer= ee.Reducer.toList() , #usar quando quer 1 linha    
    geometry = area.bounds(),    
    scale= 10)
  return pd.DataFrame(ccdc_study_area.getInfo()) # Cria o Data Frame com a informacao de todos os pixeis

Na função ccdcReducer a seguir é feita a explosão do output do CCDC para obter a dataframe em que cada coluna corresponde a um elemento do output do CCDC.

In [ ]:
def ccdcReducer(pointGeom, ccdcResult, dataFrame, endSerie, buf = False):
  """
  Entrada:
  pointGeom - Geometria do Ponto
  ccdcResult - O resultado do CCDC
  dataFrame - O Data Frame que será sempre atualizado
  endSerie - data de final da serie (truncagem)
  buf - Caso desejar os valores ao redor do ponto
  Saida:
  Data Frame atualizado
  """  
  if buf:
    areaEstudo = pointGeom.geometry().buffer(buf)
  else:
    areaEstudo = pointGeom.geometry()
  # desta forma sempre será gerada a coluna com listas, que será explodida abaixo
  ccdc_study_area = ccdcResult.reduceRegion(    
    reducer= ee.Reducer.toList(),    
    geometry = areaEstudo,    
    scale= 10)
  
  # converte o 'raster CCDC' em data frame, cada célula torna-se uma linha
  DF_study = pd.DataFrame(ccdc_study_area.getInfo())
  
  # adiciona as coordenadas do centróide de cada célula
  #df_Lat_Long = getLatLong(areaEstudo)
  dicLatLong = ee.Image.pixelLonLat().updateMask(ccdcResult.select('tStart').mask()).reduceRegion(
    reducer= ee.Reducer.toList(),
    geometry = areaEstudo,
    scale= 10)
  #converter para data frame
  df_Lat_Long = pd.DataFrame(dicLatLong.getInfo())
  # unir os dois DFs
  dataFrame = dataFrame.append(DF_study.join(df_Lat_Long), ignore_index = True)
  
  # Selecionar as colunas a explodir e as dos coeficientes
  tabExplode = []
  tabCoefs = []
  for c in dataFrame.columns:
    if 'coefs' in c or 'magnitude' in c or 'rmse' in c:
      tabExplode.append(c)
    if 'coefs' in c:
      tabCoefs.append(c)
  tabExplode = tabExplode + ['numObs','changeProb', 'tBreak', 'tEnd', 'tStart']
  
  dataFrame = dataFrame.explode(tabExplode)
  
  # Adicionar as colunas dos coeficientes dos breaks de cada banda
  coefsNames = ["INTP", "SLP", "COS", "SIN", "COS2", "SIN2", "COS3", "SIN3"]
  for col in tabCoefs:   
    prefixo = col.split('_')[0]
    colName = [prefixo + '_' + c for c in coefsNames]
    tempDf = pd.DataFrame(dataFrame[col].tolist(), columns=colName, index = dataFrame.index)
    dataFrame = pd.concat([dataFrame, tempDf], axis=1)  

  dataFrame['End_S'] = endSerie
  dataFrame['coord_ccdc'] = list(zip(dataFrame.latitude, dataFrame.longitude))
  dataFrame['Dist_Point'] = ''
  dataFrame['Point_Val'] = '' 

  return dataFrame

#### Extrair para multiplas datas de fim

In [ ]:
# #funcao bruno single point

# def singlePoint(lat, long, nomeImage, date_start, end, indices, pointBuffer, bandas, bandas_breakpoint, bandas_tmask, minObs, chiSquare, minYears, dateForm, Lambda, maxIter):
#   #definir o fim do período no dicionario de parametros
#   params_ImgCol['date_end'] = end
#   # criar o ponto a analisar
#   ponto = ee.Feature(ee.Geometry.Point([long, lat]))
#   # dataframes temporarios a retornar
#   dfTemp = gpd.pd.DataFrame([[lat_ponto, long_ponto]], columns = ['lat','long'])
#   gdfTemp = gpd.GeoDataFrame(dfTemp, geometry= gpd.points_from_xy(dfTemp.long, dfTemp.lat))  
#   #Roda o ccdc e obtem as distancias
#   ImCol = funcoes.getImageCollection(params_ImgCol, ponto, pointBuffer)
#   ccdc = funcoes.runCCDC(ImCol, params_ImgCol['bandas'], params_ccdc)
#   df_VAL_Temp = funcoes.ccdcReducer(ponto, ccdc, pd.DataFrame(), end, pointBuffer)
#   gdfTemp, df_VAL_Temp = funcoes.addDist(gdfTemp, df_VAL_Temp, i )
#   # retorna o GDF do ponto unico com a distancia ao ponto mais proximo e o DF do CCDC com todas a distancia as este ponto
#   return gdfTemp, df_VAL_Temp

In [ ]:
# #funcao bruno multipoint

# def multiPoint(end_dates, dfpoints, 
#                df_ccdc, df_ccdc_agregado, banda, PATH, Outputs, CCDC_Data_Frame, Originais_Modelo, TestePontosAlt, # parametros para o nome do ficheiro
#                nomeImage, date_start, indices, pointBuffer, # parametros para a imagecollection
#                bandas, bandas_breakpoint, bandas_tmask, minObs, chiSquare, minYears, dateForm, Lambda, maxIter, # parametros CCDC
#                ):
#   #criar um dataframe vazio para ser preenchido a cada iteracao de fim de serie
#   df_total = pd.DataFrame()

#   # Aqui precisamos verificar direito como estão nas funções
#   for end in end_dates:
#     #definir o fim do período no dicionario de parametros
#     params_ImgCol['date_end'] = end
#     i = 0
#     for Plat, Plon in list(dfpoints.col_lat_long):
#       prefixo = df_ccdc+'{0}_{1}_{2}.csv'.format(banda.upper(),dfpoints['id'].loc[i], end.replace('-',''))
#       ficheiro = os.path.join(PATH, Outputs, CCDC_Data_Frame, Originais_Modelo,TestePontosAlt,prefixo)
#       if os.path.exists(ficheiro):
#         # A ideia aqui seria caso os data frames existissem abrir e nao rodar o resto
#         df_VAL_CCDC = pd.read_csv(ficheiro, delimiter=';')
#         df_total = df_total.append(df_VAL_CCDC)
#       else:      
#         df_VAL_CCDC = pd.DataFrame() # Para o caso de salvar ponto a ponto
#         # cria o ponto
#         ponto = ee.Feature(ee.Geometry.Point([Plon,Plat]))

#         ImCollection = funcoes.getImageCollection(params_ImgCol, ponto, pointBuffer)

#         ccdc_result = funcoes.runCCDC(ImCollection, params_ImgCol['bandas'], params_ccdc)
        
#         df_VAL_CCDC = funcoes.ccdcReducer(ponto, cdc_result,df_VAL_CCDC, end, pointBuffer)

#         dfpoints, df_VAL_CCDC = funcoes.addDist(dfpoints, df_VAL_CCDC, i )
      
#         # deixei a identação como para salvar um ponto de cada vez      
#         df_VAL_CCDC.to_csv(ficheiro ,sep=';')     
        
#         #append no df total
#         df_total = df_total.append(df_VAL_CCDC)
#     # Verifica a existencia do ficheiro completo
#     save_name = df_ccdc_agregad + "{}.csv".format(banda.upper())
#     ficheiroCompleto = os.path.join(PATH, Outputs, CCDC_Data_Frame, Originais_Modelo,TestePontosAlt,save_name)
#     # aqui a mesma coisa, caso exista abre, caso contrario o salva
#     if os.path.exists(ficheiroCompleto):
#       df_total = pd.read_cdv(ficheiroCompleto, delimiter = ';')      
#     else:
#       df_total.to_csv(ficheiroCompleto, sep = ';')

#     return df_total

Função que cria o dataframe com a informação do CCDC. Execução demorada.

In [ ]:
#funcao multiplos fins daniel
def extractInfoMultipleEndDates(end_dates, points_gdf, output_path, params_ImgCol, params_ccdc, pointBuffer):

  """Cria e guarda dataframes com informação do CCDC. Podem ser entradas múltiplas datas de
  fim de série.
  
  Entrada:
   - end_dates: lista contendo as datas de truncagem da serie do CCDC
   - points_gdf: data frame com as coordenadas dos pontos de analise
   - output_path: string com a pasta onde deseja salvar os ficheiros gerados
   - params_ImgCol: dicionario com os parametros para coletar a Image Collection
   - params_ccdc: dicionario com os parametros para gerar o CCDC
   - pointBuffer: inteiro com o valor do buffer ao redor do ponto
   Saida:
    - df_total: data frame com a informacao do ccdc para todos os pontos de analise  
  """

  #guardar date_end original
  original_date_end = params_ImgCol['date_end']

  for end in end_dates:
    #criar um dataframe vazio para ser preenchido a cada iteracao de fim de serie
    df_total = pd.DataFrame() # Este DF precisa ser zerado a cada nova data pra nao ter duplicacoes

    #atualizar o fim do período no dicionario de parametros
    params_ImgCol['date_end'] = end

    #verifica se o ficheiro completo (pontos total) já existe. Caso exista, abre o ficheiro

    save_name = "Pontos_total_{}.csv".format(fromParamsReturnName(params_ImgCol, params_ccdc, 9999, 9999, pointBuffer)).replace('LON999900000E_LAT999900000N_','')
    ficheiroCompleto = os.path.join(output_path,save_name)
    if os.path.exists(ficheiroCompleto):
      print('Dataframe file {} found in storage. Reading from drive'.format(save_name))
      df_total = pd.read_csv(ficheiroCompleto, delimiter = ';')
      continue

    print('Creating dataframe...')

    i = 0
    for Plat, Plon in list(points_gdf.col_lat_long):
      
      #check if file already exists
      filename = fromParamsReturnName(params_ImgCol, params_ccdc, Plon, Plat, pointBuffer)
      ficheiro = os.path.join(output_path, filename + '.csv')

      if os.path.exists(ficheiro):
        # A ideia aqui seria caso os data frames existissem abrir e nao rodar o resto
        df_VAL_CCDC = pd.read_csv(ficheiro, delimiter=';')
        df_total = df_total.append(df_VAL_CCDC)

      else:      
        df_VAL_CCDC = pd.DataFrame() # Para o caso de salvar ponto a ponto
        # cria o ponto     
        ponto = ee.Feature(ee.Geometry.Point([Plon,Plat]))

        # Coleta o conjunto de Imagens Sentinel 2
        ImCollection = getImageCollection(params_ImgCol, ponto, buf=pointBuffer)

        #run ccdc
        ccdc_result = runCCDC(ImCollection, params_ImgCol['bandas'], params_ccdc)
        
        # Extrai o CCDC para o Ponto ou Para o Raio ao Redor do Ponto - É esta funcao que demora de ser executada
        df_VAL_CCDC = ccdcReducer(pointGeom = ponto, ccdcResult = ccdc_result,
                                          dataFrame = df_VAL_CCDC, endSerie = end,
                                          buf = pointBuffer)       

        # AQUI ENTRAR A FUNCAO NOVA DE DISTANCIA PARA TESTAR - aparentemente ok, testar com todos
        # converte o DF para GDF para poder utilizara as distancias entre as coordenadas
        # df_VAL_CCDC['coord_ccdc'] = list(zip(df_VAL_CCDC.latitude, df_VAL_CCDC.longitude))
        # df_VAL_CCDC['coord_ccdc'] = df_VAL_CCDC['coord_ccdc'].astype(str)
        # gdf_VAL_CCDC = gpd.GeoDataFrame(df_VAL_CCDC, geometry=gpd.points_from_xy( df_VAL_CCDC.longitude, df_VAL_CCDC.latitude), crs= 'EPSG: 4326' )
        # gdf_VAL_CCDC.to_crs(crs = 'EPSG: 3763', inplace = True)

        # aplica as distancias aos DFs
        points_gdf, df_VAL_CCDC = addDist(points_gdf, df_VAL_CCDC, i)
        # gdf_VAL_CCDC_dist = ckdnearest(gdf_VAL_CCDC, points_gdf) #novo join
        
        # deixei a identação como para salvar um ponto de cada vez      
        #df_VAL_CCDC.to_csv(ficheiro ,sep=';') # salva um df para cada ponto
        
        #append no df total
        df_total = df_total.append(df_VAL_CCDC)
        # df_total = df_total.append(gdf_VAL_CCDC_dist) # novo join

      i += 1
    
    # Verifica a existencia do ficheiro completo
    #save_name = "Pontos_total_{}.csv".format(fromParamsReturnName(params_ImgCol, params_ccdc, 9999, 9999, pointBuffer)).replace('LON999900000E_LAT999900000N_','')
    #ficheiroCompleto = os.path.join(output_path,save_name)
    # aqui a mesma coisa, caso exista abre, caso contrario o salva
    #if os.path.exists(ficheiroCompleto):
      #df_total = pd.read_csv(ficheiroCompleto, delimiter = ';')
      #pass
    #else:
      #df_total.to_csv(ficheiroCompleto, sep = ';')
    df_total.to_csv(ficheiroCompleto, sep = ';')

  #restaurar date_end para o valor original
  params_ImgCol['date_end'] = original_date_end
  
  return df_total, ficheiroCompleto # Precisei colocar para retornar o nome do ficheiro criado também, se não perdemos essa info.

####convertDate (pequena função auxiliar)

In [ ]:
def convertDate(data):
  """Obtem ano, mês e dia a partir de data no formato YYYY-MM-DD"""
  data = data.split('-')
  y = int(data[0])
  m = int(data[1])
  d = int(data[2])
  return y,m,d

####Função para o Join Espacial entre o dataframe do CCDC e a informação vetorial da DGT

In [ ]:
def spatialJoin(pathPoligonosDGT, dfCCDC):
  """
  Realizar o spatial join entre o dataframe do CCDC e os poligonos com alteracoes identificadas pela DGT
  Entrada:   
   - pathPoligonosDGT: String com o caminho completo dos poligonos desenhados pela DGT
   - pathDataFrameCCDC: Data Frame filtrado do CCDC
   Saida:
   """
  # 1) ABRIR OS ARQUIVOS
  ## Poligonos DGT
  gdfVal = gpd.read_file(pathPoligonosDGT)
  gdfVal.to_crs(crs = 'EPSG: 3763', inplace = True) # Originalmente eles estao em WGS84 29N converte para ETRS
  ## Pontos ISA
  
  # 2) CONVERTER O DF PARA GEO DF
  gdfCCDC = gpd.GeoDataFrame(dfCCDC, geometry = gpd.points_from_xy(dfCCDC.longitude, dfCCDC.latitude), crs = 4326 )
   
  ## criar a bordadura
  ###idBord = identity.copy() # cria uma copia do identity gerado acima
  idBord = gdfVal.copy()
  idBord['geometry'] = idBord.geometry.buffer(-10) # reduz a geometria em 10 metros
  idBord.drop(list(idBord.columns)[:-1], axis = 1, inplace = True) #remove todas as colunas menos a da geometria
  idBord['bordadura'] = 1 # cria uma nova coluna para poder identificar a borda dura
  ## novo identity para termos a area da borda dura
  
  ###identity = gpd.overlay(identity, idBord, how='identity')
  identity = gpd.overlay(gdfVal, idBord, how = 'identity')
  
  # Como o poligono inicial nao tinha a coluna de bordadura, há feições onde
  # temos 1 e Nulos, com a linha abaixo invertemos o campo onde era Nullo passa a True
  # e onde era 1 passa para False, ou 1 e 0
  identity.bordadura = identity.bordadura.isnull() 
  # Convertemos o resultado para WGS84
  identity.to_crs(crs = 'EPSG: 4326', inplace = True)
  ## As datas da DGT estao no formato (20200103) e precisam ser convertidas
  for dataCol in ['data_0', 'data_1', 'data_2', 'data_3']:      
      # primeiro converter para datetime
      maskZero = pd.Series(np.zeros(len(identity),dtype=bool))
      erro = identity[dataCol].isnull()
      identity.loc[erro, dataCol] = 0
      # converter tudo para inteiros e onde for 0 indicar 1970
      identity[dataCol] = identity[dataCol].astype(int)
      maskZero = identity.loc[:, dataCol] == 0
      identity.loc[maskZero, dataCol] = 19700101
      # converter para datetime  
      identity[dataCol] = pd.to_datetime(identity[dataCol], format = '%Y%m%d')
      identity.loc[maskZero, dataCol] = np.nan

  # 4) SPATIAL JOIN ENTRE OS CENTROIDES DO CCDC COM OS BUFFERS DE 200 METROS
  subset = gpd.sjoin(gdfCCDC, identity, how='inner')
  subset.reset_index(inplace = True)
  subset['buffer_ID'] = subset.buffer_ID.astype('int')
    
  """
  Descobrir quais linhas precisam ser duplicadas.
  Pressupondo que não é possível ter informação da 'data_3' sem existir a 'data_1'
  é possível filtrar e verificar a negação de quais dados são nulos e depois somar
  o reultado.
  0 = False False: não há data_1 e nem data_3
  1 = True  False: existe data_1 e não data_3
  2 = True  True: existem data_1 e Data_3  
  """
  cond = ~subset.filter(items=['data_1', 'data_3']).isnull()
  subset['analistas'] = cond.sum(axis=1)
  subset.loc[subset['analistas'] == 0, 'exists_event'] = False # Analista nao identificou nada
  subset.loc[subset['analistas'] > 0, 'exists_event'] = True # Analista identificou alteracao
  
  """
  CRIA UM DF TEMPORARIO PARA COPIAR AS LINHAS ONDE EXISTEM A 'DATA_3' E INSERE ESTA DATA NO CAMPO 'DATA1_Z'
  DEPOIS ADICIONA ISTO AO DATA FRAME ORIGINAL
  """ 
  subset['data1_z'] = ''
  # criar coluna para as datas anteriores
  # subset['data0_z'] = '' 
  subset['nome'] = '' # teste para nomear os analistas
  subset['tipo'] = ''
  subset['classeAnterior'] = '' 
  subset['classeAtual'] = ''   
  dfTemp = pd.DataFrame(columns = subset.columns)
  for row in subset.itertuples():
    # verifica se há duas datas e duplica a linha
    if row.analistas == 2:
        dfTemp = dfTemp.append(subset[subset.index == row.Index], ignore_index=False) 
  dfTemp.data1_z = dfTemp.data_3
  # capturar o valor da data_2
  # defTemp.data0_z = dfTemp.data_2
  dfTemp.nome = 'B' # teste para nomear os analistas
  dfTemp.tipo = dfTemp.tipo_2
  dfTemp.classeAtual = dfTemp.classe_3
  dfTemp.classeAnterior = dfTemp.classe_2

  subset.data1_z = subset.data_1
  # capturar o valor da data_0
  # subset.data0_z = subset.data_0
  subset.nome = 'A' # teste para nomear os analistas
  subset.tipo = subset.tipo_1
  subset.classeAtual = subset.classe_1
  subset.classeAnterior = subset.classe_0

  subset = subset.append(dfTemp, ignore_index=False)

  # Contagem do numero de breaks
  subset['Valid_breaks'] = np.ceil(subset.groupby(['coord_ccdc', 'nome'])['changeProb'].transform('sum'))

  # COLUNA DO DELTA MIN
  subset['delta_min'] = (subset.data1_z - subset.tBreak).dt.days
  subset.drop(['data_1', 'data_3', 'tipo_1', 'tipo_2','classe_0', 'classe_1','classe_2', 'classe_3'], axis = 1, inplace = True)  

  # verificar quais colunas tem magnitude de indices
  mags = [ t for t in subset.columns if 'magnitude' in t and not 'B' in t] 
  ordem = [ 'coord_ccdc','buffer_ID', 'IDCCDC', 'altera', 'changeProb'] + mags + ['tBreak', 'data1_z',
         'bordadura', 'classe2018', 'classe2019', 'classe2020','classe2021', 'classeAnterior','tipo',
         'classeAtual', 'analistas', 'nome', 'exists_event', 'Valid_breaks' , 'delta_min', 'geometry']
    

  return subset[ordem], subset

####Função de Validação 

In [ ]:
# função para realizar a limpeza de linhas indesejadas
def testeRemove(groupedby): 
  min_delta_min = groupedby['Min_delta_min'].min()
  #remove rows only if there is more than 1 row per point, the number of analyst dates is not zero and min_delta_min is greater than zero.
  if len(groupedby) > 1 and groupedby.analistas.min() > 0 and min_delta_min > 0:
      Bj, Ai = groupedby.loc[groupedby['delta_min']==min_delta_min][['tBreak','data1_z']].values[0]
      #remove rows that contain Ai or Bj (other than the row with the min_delta_min)
      mask = ((groupedby['tBreak'] == Bj) | (groupedby['data1_z'] == Ai)) & (groupedby['delta_min']!=min_delta_min)
      groupedby = groupedby[~mask]

  return groupedby

# função de validação do data frame
def valPol(df, theta):
  """
  Esta função recebe o geodataframe gerado no spatialJoin() e contabiliza as métricas de positivos e negativos.
  A Saída é a matriz com os cálculos e um dicinário com as métricas contabilizadas. 
  
  """
  
  # transforma a coluna de delta min para valor absoluto e cria uma nova coluna com o mínimo delta min por ponto
  df.reset_index(inplace = True)
  original_delta_min = df['delta_min'].copy()
  df['delta_min'] = abs(df['delta_min'].fillna(99999)) # substitui os nullos para evitar que sejam os minimos
  df['Min_delta_min'] = df.groupby(['coord_ccdc', 'nome'])['delta_min'].transform('min') # calcula o valor minimo por ponto
  df['delta_min'] = abs(original_delta_min) # retorna o valor absoluto da coluna original
  df['Min_delta_min'] = df['Min_delta_min'].replace(99999,np.nan) # substitui os 99999 por nullos

  bf = df.copy()

  bf['Valid_breaks'] = bf.groupby(['coord_ccdc', 'nome']).transform('count')[['tBreak']] # verifica os breaks validos por pontos
  # SE O TBREAK FOR OBJETO ELE JAMAIS SERA NULO, CONVERTER PARA DATA.
  bf.tBreak = pd.to_datetime(bf.tBreak)
  bf.tStart = pd.to_datetime(bf.tStart)
  bf.tEnd = pd.to_datetime(bf.tEnd)
  bf.analistas = bf.analistas.astype(int)
  bf.exists_event = bf.exists_event.astype(int)
  bf.buffer_ID = bf.buffer_ID.astype(int)
  bf.IDCCDC = bf.IDCCDC.astype(int)

  ## ALGUMAS MASCARAS INICIAIS NECESSARIAS
  # mascara dos breaks a mais que analistas ainda em reformulacao

  # PARA O CASO DE TER SOMENTE UM BREAK FP E DOIS ANALISTAS PARA NAO TER DUPLICACAO
  mask = pd.Series(np.zeros(len(bf),dtype=bool), index= bf.index)
  mask.loc[(bf.analistas == 2) & (bf.Valid_breaks < bf.analistas) ] = True #& (bf.delta_min > theta) 
  
  bf.loc[mask, 'Min_delta_min'] = bf.loc[mask].groupby(['coord_ccdc'])['delta_min'].transform('min')

  # Contabilizar 
  # colocar todos os VP (delta_min <=31)
  #VP
  bf.loc[( (bf.delta_min <= theta) & (~bf.tBreak.isnull()) & (bf.analistas > 0) ), 'VP'] = 1
  # #FP
  # # sem a condição da magnitude ou (changeProb ==1) serao selecionados os que devem ser negativos
  # bf.loc[( (bf.analistas == 0) & (bf.ndvi_magnitude != 0) & (~bf.tBreak.isnull())), 'FP' ] = 1 #FP puro
  # bf.loc[( (bf.delta_min > theta) & (bf.ndvi_magnitude != 0) & ( (bf.delta_min == bf.Min_delta_min) & (~bf.Min_delta_min.isnull()) ) )  , 'FP' ] = 1
  # bf.loc[( (bf.delta_min > theta) & (bf.ndvi_magnitude != 0) & (bf.analistas == 1)  ) & (~bf.tBreak.isnull()), 'FP' ] = 1
  #FP
  # sem a condição da magnitude ou (changeProb ==1) serao selecionados os que devem ser negativos
  bf.loc[( (bf.analistas == 0) & (~bf.tBreak.isnull())), 'FP' ] = 1 #FP puro
  bf.loc[( (bf.delta_min > theta) & ( (bf.delta_min == bf.Min_delta_min) & (~bf.Min_delta_min.isnull()) ) )  , 'FP' ] = 1
  bf.loc[( (bf.delta_min > theta) & (bf.analistas == 1)  ) & (~bf.tBreak.isnull()), 'FP' ] = 1
  #FN
  bf.loc[( (bf.analistas > 0)  & (bf.tBreak.isnull()) ), 'FN' ] = 1 # FN puro
  # falsos negativos que precisam ser contabilizado para os FPs
  bf.loc[(bf.analistas == 1) & (bf.Valid_breaks == 1) & (bf.FP == 1), 'FN'] = 1 # parece funcionar
  bf.loc[(bf.analistas == 2) & (bf.Valid_breaks == 3) & (bf.FP == 1) , 'FN'] = 1
  
  #VN
  bf.loc[( (bf.analistas == 0) & (bf.tBreak.isnull()) ), 'VN' ] = 1

  # converter os NaN para 0
  bf[['VP', 'FP', 'FN', 'VN']] = bf[['VP', 'FP', 'FN', 'VN']].fillna(0)

  # verificar os breaks que nao foram classificados
  # para isso gero uma coluna total onde somo todas as metricas, as linhas onde ha 0 nao foram classificadas
  bf['total'] = bf.VP + bf.FP +bf.FN + bf.VN
  mask = pd.Series(np.zeros(len(bf),dtype=bool), index= bf.index) #mascara
  # agrupar por coordenada e t break, assim as somente os breaks que nao foram validados para nenhum analista terao valor 0
  mask.loc[(bf.groupby(['coord_ccdc','tBreak'])['total'].transform('sum')==0) & (bf.analistas == 2) & (bf.Valid_breaks > bf.analistas)] = True
  # neste grupo selecionado devo procurar aquele que tem menor distancia para um analista e classificar como FP
  mask2 = bf[mask].groupby(['coord_ccdc'])['delta_min'].transform('min') == bf.delta_min[mask]
  # agora classificar os candidatos que atendem as duas mascaras
  bf.loc[(mask & mask2), ['FP']] = 1

  # Ajuste FN
  # se for na célula anterior isso contará para o total e a mascara anterior não será feita em alguns pontos onde deve ser feita
  bf.loc[((bf.FP ==1) & (bf.analistas == 1) & (bf.delta_min == bf.Min_delta_min) & (bf.Valid_breaks == 2))   , 'FN' ] = 1
  bf.loc[((bf.FP ==1) & (bf.analistas == 1) & (bf.delta_min == bf.Min_delta_min) & (bf.Valid_breaks == 3))   , 'FN' ] = 1 
  bf.loc[(bf.analistas == 2) & (bf.Valid_breaks == 1) & (bf.VP == 0), 'FN'] = 1
  bf.loc[(bf.analistas == 2) & (bf.Valid_breaks == 2) & (bf.FP == 1), 'FN'] = 1

  # Bloco para corrigir o problema de quando as duas datas DGT estão mais próximas do mesmo break
  # listar as coordenadas que tem o problema com mesmo break classificado
  listCoord = list(bf.coord_ccdc[(bf.groupby(['coord_ccdc','tBreak'])['total'].transform('sum') == 0) & (bf.analistas == 2) & (bf.Valid_breaks == 2)])
  # dividir o data frame em dois para poder limpar as linhas com problema
  bf_filter = bf.loc[~bf.coord_ccdc.isin(listCoord)].copy()
  # limpeza
  bf_remove_lines = bf.loc[bf.coord_ccdc.isin(listCoord)].copy()
  # zerar todas as métricas para poder recalcular
  bf_remove_lines.loc[:, ['VP','VN','FP', 'FN']] = 0  
  bf_removed = bf_remove_lines.groupby(['buffer_ID','IDCCDC']).apply(testeRemove).copy() # função de remoção
  bf_removed = bf_removed.drop(columns=['buffer_ID','IDCCDC']).reset_index() # evitar problema de indece dup.
  # Agora teremos somente duas linhas por ponto que são obrigatóriamente FP ou VP
  #VP
  bf_removed.loc[( (bf_removed.delta_min <= theta) ), 'VP'] = 1
  #FP, FN
  bf_removed.loc[( (bf_removed.delta_min > theta) ), ['FP', 'FN']] = 1
  # unir os dois dfs novamente
  bf_final = bf_filter.append(bf_removed)

  # remover aqueles que nao possuem metrica
  bf_final = bf_final[(bf_final.VP > 0) | (bf_final.FP > 0) | (bf_final.FN > 0) | (bf_final.VN > 0) ].copy()
  # remover aqueles que apresentam as classes especificas
  bf_final = bf_final[~(bf_final.tipo.isin(['Agricultura','Agua']))].copy()
  
  
  # verificar quais colunas tem magnitude de indices
  mags = [ t for t in bf_final.columns if 'magnitude' in t and not 'B' in t]
  # colunas para retornar um DF mais limpo
  c = ['buffer_ID', 'IDCCDC', 'coord_ccdc', 'changeProb'] + mags + ['tBreak',
       'data1_z', 'analistas', 'nome', 'exists_event', 'Valid_breaks', 
       'delta_min', 'Min_delta_min', 'VP', 'FP', 'FN', 'VN'] #geometry
  # também poderá retornar o DF todo classificado, em processo.
  return bf_final[c], bf_final

####Extrair informacao do raster para o ponto

In [ ]:
def getRasterValue(gdf, Raster, label):
  """
  Extrair os valores de um raster para o ponto
  Entrada - gdf: Objeto GeoDataFrame
          - Raster: string com o caminho completo do raster para extrair os valores (eg COSsim2018_H)
          - label: string para identificar os rasters
  Saida - GeoDataFrame com os valores dos rasters
  """
  raster = rasterio.open(Raster) # abre o arquivo raster
  gdf_crs = gdf.crs # salvar a coordenada original
  
  # Verifica se o raster e o geodataframe estao na mesma coordenada
  if gdf.crs != raster:
    gdf = gdf.to_crs(raster.crs)
  else:
    pass
  
  # coletar os pares de coordenadas x, y 
  coords = [(x,y) for x,y in zip(gdf.geometry.x, gdf.geometry.y)]
  gdf[label] = [x[0] for x in raster.sample(coords)]
  
  # Verificar as coordenadas de retorno
  if gdf.crs != gdf_crs:
    gdf = gdf.to_crs(gdf_crs)
    return gdf
  else:
    return gdf

####Transformar em matriz binária

In [ ]:
def dfBinario_v2(DF_FINAL):
  """
  Converte a data frame para matriz binária
  Entrada: Data Frame completa com toda a informacao
  Saida: nova Data Frame com os valores binarios
   
  Classes de uso do solo indentificados pela DGT
  PBR: Pinheiro bravo; MAT: Matos; VHE: Vegetacao herbace espontanea; EUC: Eucalipto;
  OFP: Outras folhosas persistente; AGR: Agricultura; SVE: Superficie sem vegetacao escura;
  SVC: Superficie sem vegetacao clara; 
  OFC: Outras folhosas caduca; SEA: Sobreiro e Azinheira; PMA: Pinheiro Manso

  tipo de alteracao identificada pela DGT
  TIP_AGR:  Agricultura, TIP_AGU: Agua, TIP_FOG:Fogo, TIP_COR: Corte

  Classes de uso do solo da COSSIM
  100: Artificializado; 200: Agricultura (somente COSSIM 18); 211: Culturas anuais de outono inverno;
  212: Culturas anuais de primavera verao; 213: Outras areas agrucilas; 311: Sobreiro e Azinheira;
  312: Eucalipto; 313: Outras folhosas; 321: Pinheiro bravo; 322: Pinheiro manso, 323: Outras reinosas;
  410: Matos; 420: Vegetacao herbacea espontanea; 500: Supercies sem vegetacao; 610: Zonas humidas;
  620: Agua
  """

  sub = DF_FINAL.copy()
  new = pd.DataFrame(index= sub.index)

  dicTipo = {'Agricultura': 'TIP_AGR', 'Agua': 'TIP_AGU', 'Fogo': 'TIP_FOG', 'Corte': 'TIP_COR'}

  dicCossim = {100: 'ART', 200: 'AGR', 211: 'AGR', 212: 'AGR', 213: 'AGR', 311: 'SEA', 312: 'EUC', 313: 'OFS',
          321: 'PBR', 322: 'PMA', 323: 'ORS', 410: 'MAT', 420: 'VHE', 500: 'SVG',610: 'ZHU', 620: 'AGU'}

  dicClasse = {'Superficie sem vegetacao escura': 'SVE', 'Pinheiro bravo': 'PBR', 'Pinheiro manso': 'PBA',
              'Matos': 'MAT', 'Vegetacao herbacea espontanea': 'VHE', 'Eucalipto': 'EUC', 'Outras folhosas persistente': 'OFP',
              'Outras folhosas caduca': 'OFC', 'Sobreiro e Azinheira': 'SEA', 'Agricultura': 'AGR', 'Superficie sem vegetacao clara': 'SVC' }

  # Converter as colunas de datas para numero de dias apos 01/01/2016.
  sub['tStart'] =  (sub.tStart - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days # dara do inicio do segmento
  sub['breakMS'] = (sub.tBreak - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days # data do break
  sub['tEnd'] = (sub.tEnd - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days # data do final do segmento
  sub['dgtMS'] = (sub.data1_z - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days # data da alteracao do analista
  #sub['dgtMS_ant'] = (sub.data0_z - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days # data da imagem interior a alteracao do analista

  sub.loc[sub['altera'] == 'Com Alteracao', 'altera'] = 1
  sub.loc[sub['altera'] == 'Sem Alteracao', 'altera'] = 0

  for coluna in sub.columns:
    if 'tipo' in coluna: # converte os valores do 'TIPO' de ocorrencia identificada pela DGT para a alteracao
      for kt in dicTipo.keys():
        new[dicTipo[kt]] = 0
        new.loc[sub[coluna] == kt, dicTipo[kt]] = 1

    if 'classe' in coluna: # converte os valores das 'CLASSES' de ocupacao indetificada pela DGT
      for kc in dicClasse.keys():
        novaColC = dicClasse[kc] + '_' + coluna.replace('classe', '')
        new[novaColC] = 0
        new.loc[sub[coluna] == kc, novaColC] = 1

    if 'COSSIM' in coluna: # converte os valores das 'COSSIMs' originais
      for k in dicCossim.keys():
        novaCol = dicCossim[k] + '_' + coluna      
        new[novaCol] = 0
        new.loc[sub[coluna] == k, novaCol] = 1  
  # colunas desnecessarias na matriz binaria
  remove = ['buffer_ID', 'IDCCDC', 'altera', 'changeProb', 'tBreak', 'classe2018', 'classe2019', 'classe2020',
            'classe2021','coord_ccdc', 'data1_z', 'tipo', 'classeAnterior', 'classeAtual', 'Min_delta_min',
            'delta_min', 'Valid_breaks', 'nome', 'analistas', 'notas', 'area', 'data_2',
              'data_0', 'ID', 'index_right',  'breaks_in_tmask', 'numBreak', 'startMin', 'Point_Val',
              'Dist_Point', 'End_S', 'numObs', 'level_0', 'index', 'latitude', 'longitude', 'total', 'level_2']  
  for c in sub.columns: # verifica as colunas com cossim ou coef no nome para eliminar
    if '_coefs' in c or 'COSSIM' in c: 
      remove.append(c)

  subreturn = sub.drop(remove, axis=1) # Remove as colunas que serao desnecessarias
  #subreturn = sub[ordem].copy() # A parte da ordem pode ser utilizada para selecionar as colunas que queira se retornar
  subreturn.bordadura = subreturn.bordadura.astype(int)
  subreturn.exists_event = subreturn.exists_event.astype(int)

  return pd.concat([subreturn,new], axis=1)

Função antiga, substituida pela anterior acima

In [ ]:
def dfBinario(sub):
  # Converter as colunas de datas para numero de dias apos 01/01/2016.
  sub['tStart'] =  (sub.tStart - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days 
  sub['breakMS'] = (sub.tBreak - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days
  sub['tEnd'] = (sub.tEnd - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days
  sub['dgtMS'] = (sub.data1_z - pd.to_datetime(datetime.datetime(2016,1,1))).dt.days

  """
  Classes de uso do solo indentificados
  PBR: Pinheiro bravo; MAT: Matos; VHE: Vegetacao herbace espontanea; EUC: Eucalipto;
  OFP: Outras folhosas persistente; AGR: Agricultura; SVE: Superficie sem vegetacao escura;
  SVC: Superficie sem vegetacao clara; 
  OFC: Outras folhosas caduca; SEA: Sobreiro e Azinheira; PMA: Pinheiro Manso

  tipo de alteracao identificada
  TIP_AGR:  Agricultura, TIP_AGU: Agua, TIP_FOG:Fogo, TIP_COR: Corte
  """
  
  nCol = ['PBR', 'PMA', 'MAT', 'VHE', 'EUC', 'OFP','OFC', 'SEA', 'AGR', 'SVE', 'SVC']
  tipos = ['TIP_AGR', 'TIP_AGU', 'TIP_FOG', 'TIP_COR']
  cossims = ['ART', 'AGR', 'SEA', 'EUC', 'OFS','PBR', 'PMA', 'ORS', 'MAT', 'VHE', 'SVG', 'AGU']
  nColA = []
  n18 = []
  n19 = []
  n20 = []
  n21 = []
  cossim2018 = []
  cossim2020 = []
  cossim2021 = []
  for i in nCol:
    nColA.append(i+ '_ANT')
    n18.append(i + '_DGT2018')
    n19.append(i + '_DGT2019')
    n20.append(i + '_DGT2020')
    n21.append(i + '_DGT2021')
  for c in cossims:
    cossim2018.append(c + '_COSsim18_H')
    cossim2020.append(c + '_COSsim20_H')
    cossim2021.append(c + '_COSsim21_H')

  sub[tipos] = 0
  sub[nColA] = 0
  sub[nCol] = 0
  sub[n18] = 0
  sub[n19] = 0
  sub[n20] = 0
  sub[n21] = 0
  sub[cossim2018]=0
  sub[cossim2020]=0
  sub[cossim2021]=0

  sub.loc[sub['altera'] == 'Com Alteracao', 'altera'] = 1
  sub.loc[sub['altera'] == 'Sem Alteracao', 'altera'] = 0

  sub.loc[sub['tipo'] == 'Agricultura', 'TIP_AGR'] = 1
  sub.loc[sub['tipo'] == 'Agua', 'TIP_AGU'] = 1
  sub.loc[sub['tipo'] == 'Fogo', 'TIP_FOG'] = 1
  sub.loc[sub['tipo'] == 'Corte', 'TIP_COR'] = 1

  sub.loc[sub['classeAtual'] == 'Pinheiro bravo', 'PBR'] = 1
  sub.loc[sub['classeAtual'] == 'Pinheiro manso', 'PMA'] = 1
  sub.loc[sub['classeAtual'] == 'Matos', 'MAT'] = 1
  sub.loc[sub['classeAtual'] == 'Vegetacao herbacea espontanea', 'VHE'] = 1
  sub.loc[sub['classeAtual'] == 'Eucalipto', 'EUC'] = 1
  sub.loc[sub['classeAtual'] == 'Outras folhosas persistente', 'OFP'] = 1
  sub.loc[sub['classeAtual'] == 'Agricultura', 'AGR'] = 1
  sub.loc[sub['classeAtual'] == 'Superficie sem vegetacao escura', 'SVE'] = 1
  sub.loc[sub['classeAtual'] == 'Superficie sem vegetacao clara', 'SVC'] = 1
  sub.loc[sub['classeAtual'] == 'Outras folhosas persistente', 'OFC'] = 1
  sub.loc[sub['classeAtual'] == 'Sobreiro e Azinheira', 'SEA'] = 1

  sub.loc[sub['classeAnterior'] == 'Pinheiro bravo', 'PBR_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Pinheiro manso', 'PMA_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Matos', 'MAT_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Vegetacao herbacea espontanea', 'VHE_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Eucalipto', 'EUC_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Outras folhosas persistente', 'OFP_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Agricultura', 'AGR_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Superficie sem vegetacao escura', 'SVE_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Superficie sem vegetacao clara', 'SVC_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Outras folhosas persistente', 'OFC_ANT'] = 1
  sub.loc[sub['classeAnterior'] == 'Sobreiro e Azinheira', 'SEA_ANT'] = 1

  sub.loc[sub['classe2018'] == 'Pinheiro bravo', 'PBR_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Pinheiro manso', 'PMA_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Matos', 'MAT_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Vegetacao herbacea espontanea', 'VHE_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Eucalipto', 'EUC_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Outras folhosas persistente', 'OFP_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Agricultura', 'AGR_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Superficie sem vegetacao escura', 'SVE_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Superficie sem vegetacao clara', 'SVC_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Outras folhosas persistente', 'OFC_DGT2018'] = 1
  sub.loc[sub['classe2018'] == 'Sobreiro e Azinheira', 'SEA_DGT2018'] = 1

  sub.loc[sub['classe2019'] == 'Pinheiro bravo', 'PBR_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Pinheiro manso', 'PMA_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Matos', 'MAT_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Vegetacao herbacea espontanea', 'VHE_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Eucalipto', 'EUC_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Outras folhosas persistente', 'OFP_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Agricultura', 'AGR_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Superficie sem vegetacao escura', 'SVE_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Superficie sem vegetacao clara', 'SVC_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Outras folhosas persistente', 'OFC_DGT2019'] = 1
  sub.loc[sub['classe2019'] == 'Sobreiro e Azinheira', 'SEA_DGT2019'] = 1

  sub.loc[sub['classe2020'] == 'Pinheiro bravo', 'PBR_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Pinheiro manso', 'PMA_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Matos', 'MAT_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Vegetacao herbacea espontanea', 'VHE_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Eucalipto', 'EUC_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Outras folhosas persistente', 'OFP_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Agricultura', 'AGR_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Superficie sem vegetacao escura', 'SVE_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Superficie sem vegetacao clara', 'SVC_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Outras folhosas persistente', 'OFC_DGT2020'] = 1
  sub.loc[sub['classe2020'] == 'Sobreiro e Azinheira', 'SEA_DGT2020'] = 1

  sub.loc[sub['classe2021'] == 'Pinheiro bravo', 'PBR_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Pinheiro manso', 'PMA_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Matos', 'MAT_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Vegetacao herbacea espontanea', 'VHE_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Eucalipto', 'EUC_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Outras folhosas persistente', 'OFP_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Agricultura', 'AGR_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Superficie sem vegetacao escura', 'SVE_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Superficie sem vegetacao clara', 'SVC_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Outras folhosas persistente', 'OFC_DGT2021'] = 1
  sub.loc[sub['classe2021'] == 'Sobreiro e Azinheira', 'SEA_DGT2021'] = 1

  sub.loc[sub['COSSIM18_H'] == 100, 'ART_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 200, 'AGR_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 311, 'SEA_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 312, 'EUC_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 313, 'OFS_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 321, 'PBR_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 322, 'PMA_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 323, 'ORS_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 410, 'MAT_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 420, 'VHE_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 500, 'SVG_COSsim18_H'] = 1
  sub.loc[sub['COSSIM18_H'] == 620, 'AGU_COSsim18_H'] = 1
  
  sub.loc[sub['COSSIM20_H'] == 100, 'ART_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] // 100 == 2, 'AGR_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 311, 'SEA_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 312, 'EUC_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 313, 'OFS_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 321, 'PBR_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 322, 'PMA_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 323, 'ORS_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 410, 'MAT_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 420, 'VHE_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 500, 'SVG_COSsim20_H'] = 1
  sub.loc[sub['COSSIM20_H'] == 620, 'AGU_COSsim20_H'] = 1
 
  sub.loc[sub['COSSIM21_H'] == 100, 'ART_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] // 100 == 2, 'AGR_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 311, 'SEA_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 312, 'EUC_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 313, 'OFS_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 321, 'PBR_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 322, 'PMA_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 323, 'ORS_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 410, 'MAT_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 420, 'VHE_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 500, 'SVG_COSsim21_H'] = 1
  sub.loc[sub['COSSIM21_H'] == 620, 'AGU_COSsim21_H'] = 1
  
  remove = ['buffer_ID', 'IDCCDC', 'altera', 'changeProb', 'tBreak', 'classe2018', 'classe2019', 'classe2020',
          'classe2021','coord_ccdc', 'data1_z', 'tipo', 'classeAnterior', 'classeAtual', 'Min_delta_min',
           'delta_min', 'Valid_breaks', 'nome', 'analistas', 'notas', 'area', 'data_2',
            'data_0', 'ID', 'index_right',  'breaks_in_tmask', 'numBreak', 'startMin', 'Point_Val',
            'Dist_Point', 'End_S', 'numObs', 'level_0', 'index', 'latitude', 'longitude', 'total', 'level_2']  
  for c in sub.columns:
    if '_coefs' in c:
      remove.append(c)

#   ordem = ['breakMS', 'tStart', 'tEnd', 'dgtMS', 'VP', 'FP', 'FN', 'VN', 'bordadura', 'exists_event', 'B11_COS', 'B11_COS2', 'B11_COS3',
#  'B11_INTP', 'B11_SIN', 'B11_SIN2', 'B11_SIN3', 'B11_SLP', 'B11_magnitude', 'B11_rmse', 'B12_COS', 'B12_COS2', 'B12_COS3',
#  'B12_INTP', 'B12_SIN', 'B12_SIN2', 'B12_SIN3', 'B12_SLP', 'B12_magnitude', 'B12_rmse','B2_COS', 'B2_COS2', 'B2_COS3', 'B2_INTP',
#  'B2_SIN', 'B2_SIN2', 'B2_SIN3', 'B2_SLP', 'B2_magnitude', 'B2_rmse','B3_COS', 'B3_COS2', 'B3_COS3', 'B3_INTP', 'B3_SIN',
#  'B3_SIN2', 'B3_SIN3', 'B3_SLP', 'B3_magnitude', 'B3_rmse', 'B4_COS', 'B4_COS2', 'B4_COS3', 'B4_INTP', 'B4_SIN', 'B4_SIN2',
#  'B4_SIN3', 'B4_SLP', 'B4_magnitude', 'B4_rmse', 'B8_COS', 'B8_COS2', 'B8_COS3', 'B8_INTP', 'B8_SIN', 'B8_SIN2', 'B8_SIN3',
#  'B8_SLP', 'B8_magnitude', 'B8_rmse','ndvi_COS', 'ndvi_COS2', 'ndvi_COS3', 'ndvi_INTP', 'ndvi_SIN', 'ndvi_SIN2', 'ndvi_SIN3',
#  'ndvi_SLP', 'ndvi_magnitude', 'ndvi_rmse', 'TIP_AGR', 'TIP_AGU', 'TIP_FOG', 'TIP_COR', 'PBR_ANT', 'MAT_ANT', 'VHE_ANT', 'EUC_ANT',
#  'OFP_ANT', 'OFC_ANT', 'SEA_ANT', 'AGR_ANT', 'SVE_ANT', 'SVC_ANT', 'PBR', 'MAT', 'VHE', 'EUC', 'OFP', 'OFC', 'SEA', 'AGR', 'SVE',
#  'SVC', 'PBR_DGT2018', 'PMA_DGT2018', 'MAT_DGT2018', 'VHE_DGT2018', 'EUC_DGT2018', 'OFP_DGT2018', 'OFC_DGT2018', 'SEA_DGT2018', 'AGR_DGT2018', 'SVE_DGT2018', 'SVC_DGT2018',
#  'PBR_DGT2019', 'PMA_DGT2019', 'MAT_DGT2019', 'VHE_DGT2019', 'EUC_DGT2019', 'OFP_DGT2019', 'OFC_DGT2019', 'SEA_DGT2019', 'AGR_DGT2019', 'SVE_DGT2019', 'SVC_DGT2019',
#  'PBR_DGT2020','PMA_DGT2020', 'MAT_DGT2020', 'VHE_DGT2020', 'EUC_DGT2020', 'OFP_DGT2020', 'OFC_DGT2020', 'SEA_DGT2020', 'AGR_DGT2020', 'SVE_DGT2020', 'SVC_DGT2020',
#  'PBR_DGT2021', 'PMA_DGT2021', 'MAT_DGT2021', 'VHE_DGT2021', 'EUC_DGT2021', 'OFP_DGT2021', 'OFC_DGT2021', 'SEA_DGT2021', 'AGR_DGT2021', 'SVE_DGT2021', 'SVC_DGT2021',
#  'ART_COSsim18_H', 'AGR_COSsim18_H', 'SEA_COSsim18_H', 'EUC_COSsim18_H', 'OFS_COSsim18_H', 'PBR_COSsim18_H', 'PMA_COSsim18_H', 'ORS_COSsim18_H', 'MAT_COSsim18_H',
#  'VHE_COSsim18_H', 'SVG_COSsim18_H', 'AGU_COSsim18_H', 'ART_COSsim20_H', 'AGR_COSsim20_H', 'SEA_COSsim20_H', 'EUC_COSsim20_H', 'OFS_COSsim20_H', 'PBR_COSsim20_H',
#  'PMA_COSsim20_H', 'ORS_COSsim20_H', 'MAT_COSsim20_H', 'VHE_COSsim20_H', 'SVG_COSsim20_H', 'AGU_COSsim20_H', 'ART_COSsim21_H', 'AGR_COSsim21_H', 'SEA_COSsim21_H',
#  'EUC_COSsim21_H', 'OFS_COSsim21_H', 'PBR_COSsim21_H', 'PMA_COSsim21_H', 'ORS_COSsim21_H', 'MAT_COSsim21_H', 'VHE_COSsim21_H', 'SVG_COSsim21_H', 'AGU_COSsim21_H',
#   'geometry']

  subreturn = sub.drop(remove, axis=1)
  #subreturn = sub[ordem].copy()
  subreturn.bordadura = subreturn.bordadura.astype(int)
  subreturn.exists_event = subreturn.exists_event.astype(int)

  return subreturn

####filtroDF - <font color='red'>ANTIGO</font>

In [ ]:
def filtroDF(validacao, DF, ti, tf):# Nao sera mais utilizada
  """
  Filtra o data frame do ccdc para o periodo de analise desejado, eliminando
  os tbreaks fora do intervalo desejado. E adiciona a informacao do ccdc nos
  dados de validacao, a partir de um join espacial para o ponto mais proximo
  Entrada:
   - validacao: data frame com os pontos da analise
   - DF: data frame com o resultado do ccdc
   - ti: string com a data inicial do periodo de analise no formato 'YYYY-MM-DD'
   - tf: string com a data final do periodo de analise no formato 'YYYY-MM-DD'
  Saida:
   - join: data frame dos pontos de analise contendo a informacao do CCDC
  """
  
  colunasFiltro = ['altrc1_z', 'tipo1_z', 'data1_z','GC_z', 'OBID', 'COS2018_Lg', 'ndvi_magnitude',
                   'ndvi_rmse','tBreak', 'tEnd', 'tStart', 'geometry', 'End_S' , 'coord_ccdc']
  
  # Transformar as colunas em Date Time
  for dtCol in DF.columns:
    if 'tBreak' in dtCol or 'tEnd' in dtCol or 'tStart' in dtCol:
      mask = DF.loc[:, dtCol] == 0
      DF[dtCol] = pd.to_datetime(DF[dtCol], unit = 'ms')
      DF.loc[mask, dtCol] = np.nan
    elif 'End_S' in dtCol:
      DF[dtCol] = pd.to_datetime(DF[dtCol])
  # filtro das datas
  yi, mi, di = convertDate(ti)
  fltInicial = datetime.datetime(yi, mi, di)
  yf, mf, df = convertDate(tf)
  fltFinal = datetime.datetime(yf, mf, df)
  if fltFinal > datetime.datetime(2020,7,28):
    fltFinal = datetime.datetime(2020,7,28)

  mask = pd.Series(np.zeros(len(DF),dtype=bool))
  for dtCol in DF.columns:
    if 'tBreak' in dtCol:
      # esta condicao garante que somente as datas entre o intervalo selecionado não serão null
      cond=np.array(DF[dtCol] >= fltInicial) & np.array(DF[dtCol] <= fltFinal)
      mask.loc[cond] = True
      DF.loc[~mask, dtCol] = np.nan
  
  join = pd.merge(validacao, DF[(DF['tBreak'] >= fltInicial) & (DF['tBreak'] <= fltFinal)], on= 'coord_ccdc', how='left')# OBID era no local do geometry
  for c in join.columns:
    if c not in colunasFiltro:
      join.drop(c, axis = 1, inplace=True)
  
  if 'data1_z' in join.columns:
    # AQUI APLICA O FILTRO DE DATAS TAMBÉM PARA O Z, CASO O CONTRARIO TEREMOS MAIS FALSOS NEGATIVOS
    # primeiro converter para datetime
    maskZero = pd.Series(np.zeros(len(join),dtype=bool))
    erro = join['data1_z'].isnull()
    join.loc[erro, 'data1_z'] = 0
    # converter tudo para inteiros e onde for 0 indicar 1970
    join['data1_z'] = join['data1_z'].astype(int)
    maskZero = join.loc[:, 'data1_z'] == 0
    join.loc[maskZero, 'data1_z'] = 19700101
    # converter para datetime  
    join['data1_z'] = pd.to_datetime(join['data1_z'], format = '%Y%m%d')
    join.loc[maskZero, 'data1_z'] = np.nan
    # aplicar o filtro
    cond = np.array(join['data1_z'] >= fltInicial) & np.array(join['data1_z'] <= fltFinal)
    maskTime = pd.Series(np.zeros(len(join),dtype=bool))
    maskTime.loc[cond] = True
    join.loc[~maskTime, 'data1_z'] = np.nan
  
  return join 
  

####Funções para validação - <font color='red'>ANTIGAS</font>

In [ ]:
#NAO UTILIZADAS
# the input is a Series of deltatimes, that can have positive or negative values
def extreme_deltatimes(dtseries,mytype):
  dtseries = dtseries[dtseries.notnull()] # remove not a time NaT
  dts = dtseries.dt.days.unique() # returns numpy array
  if mytype == 'min':
    return dts[np.argmin(abs(dts))] # deltamin  

# myseries(Series) is one row of a DataFrame 
# returns either deltamin(integer) or deltamax(integer) of myseries
def mydeltas(myseries, tbreakcolumns, magcolumns,eventcolumns,mytype):
  if myseries['breaks_in_tmask'] and myseries['exists_event']:   
    tBreakCol = tbreakcolumns[0]    
    tBreak = myseries.loc[tBreakCol]     
    return extreme_deltatimes(pd.to_datetime(myseries[eventcolumns]) - pd.to_datetime(tBreak),mytype)    
  else:
    return np.nan # if no break exists in the temporal filter

def MAG_colname(tBreak_colname,INDEX):
  if isinstance(tBreak_colname,type(None)):
    return None
  else:
    s = tBreak_colname.split('tBreak')
    newCol = s[0]+INDEX+'_magnitude'
    return newCol

def tBreak_colname(MAG_colname,INDEX):
  if isinstance(MAG_colname,type(None)):
    return None
  else:
    s = MAG_colname.split(INDEX+'magnitude')
    newCol = s[0]+'_tBreak'
    return newCol

def convertDataFrame(df):
  """Organizar o data frame para obtencao das metricas almejadas"""
  colNames = ['OBID','tBreak','ndvi_magnitude','breaks_in_tmask','exists_event',
              'num_events','delta_min', 'data1_z', 'tipo1_z']
  tbreakcolumns=[]
  magcolumns=[]
  # Criar as colunas com datas
  for dtCol in df.columns:
    if 'tBreak' in dtCol or 'tEnd' in dtCol or 'tStart' in dtCol or 'data1_z' in dtCol:
      mask = df.loc[:, dtCol] == 0
      df[dtCol] = pd.to_datetime(df[dtCol])
      df.loc[mask, dtCol] = np.nan  
    if 'tBreak' in dtCol: 
      if df[dtCol].count() > 0 :
        newCol = MAG_colname(dtCol,INDEX)
        tbreakcolumns.append(dtCol) # only keep columns which have non nan's
        magcolumns.append(newCol) # columns _MAG that correspond to breaks  
  mask=pd.Series(np.zeros(len(df),dtype=bool)) # initialize array of False
  num_tbreaks_por_ponto = pd.Series(np.zeros(len(df))) # initialize array of zeros

  for dtCol in tbreakcolumns:
    cond = ~np.isnan(np.array(df[dtCol]))
    mask.loc[cond] = True  
    num_tbreaks_por_ponto[cond] += 1
  
  df['breaks_in_tmask'] = mask
  df['num_breaks_in_tmask'] = pd.to_numeric(num_tbreaks_por_ponto,downcast='integer')
  
  eventcolumns = ['data1_z'] # somente contabilizar o Z
  mask = pd.Series(np.zeros(len(df),dtype=bool)) # number rows)
  num_events = pd.Series(np.zeros(len(df)))
  for dtCol in eventcolumns:
    cond = ~np.isnat(np.array(df[dtCol])) # ~ is negation element-wise
    mask.loc[cond] = True
    num_events[cond] += 1
  df['exists_event'] = mask
  df['num_events'] = num_events
  df['delta_min'] = df.apply(lambda row : mydeltas(row,tbreakcolumns, magcolumns,eventcolumns,'min'), axis = 1)
  
  return df[colNames].loc[:]

# UTILIZAÇÃO: teste = convertDataFrame(gdf)

In [ ]:
# NAO UTILIZADAS
def validacaoDataFrame(df, theta = 30):
  """
  Entrada: df: Data Frame filtrado para as datas desejadas
           theta: valor maximo de dias permitidos entre
           o modelo e o analista, default 30.
  Saida: Data Frames de VP, FP, FN e VN e matriz de confusao
  """
  colNames=['OBID','tBreak','ndvi_magnitude','exists_event','num_events',
            'delta_min', 'breaks_in_tmask','data1_z', 'tipo1_z']
  #e[1] = Z True e break True e teta < +- theta dias
  VP = df[colNames].loc[(df.breaks_in_tmask == True) & (df.exists_event==True) & (abs(df.delta_min) <= theta)]
  mask=pd.Series(np.zeros(len(df),dtype=bool))
  mask.loc[(df.breaks_in_tmask == True) & (df.exists_event==True) & (abs(df.delta_min) <= theta)]=True
  df.loc[mask, 'CCDC_data'] = 1
  df.loc[mask, 'REF_DATA'] = 1

  #e[2] = Z Falso e break = True
  FP = df[colNames].loc[(df.breaks_in_tmask == True) & (df.exists_event==False)]
  mask = pd.Series(np.zeros(len(df),dtype=bool))
  mask.loc[(df.breaks_in_tmask == True) & (df.exists_event==False)]=True
  df.loc[mask, 'CCDC_data'] = 1
  df.loc[mask, 'REF_DATA'] = 0

  #e[3] = Z True e break False (pode ser fora do tmask ou nunca ter existido ou > +/- theta) 
  # Apesar de o Modelo encontrar a quebra, como o teta é maior que o estipulado é como se o modelo tivesse errado e não encontrado a alteração
  FN = df[colNames].loc[((df.breaks_in_tmask == False) & (df.exists_event==True)) | ((df.breaks_in_tmask == True) & (abs(df.delta_min) > theta))]
  mask = pd.Series(np.zeros(len(df),dtype=bool))
  mask.loc[((df.breaks_in_tmask == False) & (df.exists_event==True)) | ((df.breaks_in_tmask == True) & (abs(df.delta_min) > theta))]=True
  df.loc[mask, 'CCDC_data'] = 0
  df.loc[mask, 'REF_DATA'] = 1

  #e[4] = Z False e break False
  VN = df[colNames].loc[(df.breaks_in_tmask == False) & (df.exists_event==False)]
  mask=pd.Series(np.zeros(len(df),dtype=bool))
  mask.loc[(df.breaks_in_tmask == False) & (df.exists_event==False)]=True
  df.loc[mask, 'CCDC_data'] = 0
  df.loc[mask, 'REF_DATA'] = 0

  confusion_matrix = pd.crosstab(df['REF_DATA'], df['CCDC_data'], rownames=['REFERENCIA'], colnames=['MODELO'])

  return VP, FP, FN, VN, confusion_matrix


# UTILIZAÇÃO: vp, fp, fn, vn, matriz = validacaoDataFrame(teste, 30)

####Adiciona as novas colunas para o Join e converte para WGS 84

In [ ]:
def dfColumnsToJoin(gdf):
  """
  Converte para a coordenada desejada e adiciona as colunas utilizadas para 
  realizar o join por Distancia.
  """
  # Converte para WGS 84
  gdf.to_crs(crs = 'EPSG: 4326', inplace = True)
  # cria uma tupla com as coordenadas
  gdf['col_lat_long'] = list(zip(gdf.geometry.values.y, gdf.geometry.values.x))
  gdf['dist_ccdc'] = '' # receberá as distancias para as células
  gdf['coord_ccdc'] = ''
  return gdf

####Verificar as distancias

In [ ]:
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(np.ceil(dist), name='distance')
        ], 
        axis=1)

    return gdf

In [ ]:
#  verifica a distancia entre os pontos
def distance_from(loc1,loc2):
  """
  Entrada:
  loc1 - tupla com Coordenada do ponto do analista (lat, long)
  loc2 - tupla com Coordenada da celula do CCDC (lat, long)
  Saida:
  dist - A distancia entre os pontos
  loc2 - Isso precisa ser inserido no Data Frame de validacao, caso contrario
  não há uma coluna igual nos dois Data Frames que possibilite fazer o join
  """ 
  dist = hs.haversine(loc1,loc2, unit=Unit.METERS)
  return (round(dist,8),loc2)

####Adiciona as distancias da celula mais proxima aos pontos de validacao e todas as distancias do ponto nas celulas do df ccdc

In [ ]:
def addDist(points, dfccdc, i): 
  """
  Verifica a distancia entre os pontos dos data frames dados de entrada
  Entrada:
   - points: data frame com os pontos de analise
   - dfccdc: data frame com o resultado do ccdc
   - i: inteiro para indicar a linha do df 'points' que se quer a distancia
   Saida:
    - data frames de entrada atualizados com todas as distancias
  """
  Plat, Plon = points.col_lat_long.loc[i]
  # cria uma lista temporaria contendo a distancia do ponto[i] para todas as
  #  coordenadas do dfccdc      
  tempList = dfccdc['coord_ccdc'].apply(lambda x: distance_from((Plat, Plon),x))
  # insere no df points a menor distancia encontrada e a coordenada do ponto
  #  mais proximo do dfccdc
  points['dist_ccdc'].loc[i] = min(tempList)[0]
  points['coord_ccdc'].loc[i] = min(tempList)[1]
  # insere no dfccdc a distancia de todas as coordenadas para o ponto[i] e 
  # as coordenadas deste ponto
  dfccdc['Dist_Point'] = dfccdc['coord_ccdc'].apply(lambda x: distance_from((Plat, Plon),x)[0])
  dfccdc['Point_Val'] = points.geometry.loc[i]  

  return points, dfccdc

####Gerar gráfico e obter o número de VPs para séries com finais variados - ANTIGO

In [ ]:
def countVPs(df, theta = 30, delta = (60, 90)):
  """
  Entrada:
   - df: Data Frame filtrado
   - theta: inteiro,0 numero de  +/- dias permitidos entre o break e o analista,
         Padrão = 30
   - delta: tupla, intervalo de dias apos a data do analista para selecionar a
         truncagem do modelo, padrão = (60,90) dias
  Saida:
   - número de verdadeiros positivos que atendem as opções
  """
  cond = np.array(((df.Dif_data >= datetime.timedelta(int(delta[0]))) & (df.Dif_data <= datetime.timedelta(int(delta[1])))))
  cond2 = np.array(((df.Dif_Z_Tbreak >= datetime.timedelta(-theta)) & (df.Dif_Z_Tbreak <= datetime.timedelta(theta))))
  df['Dif_data_Bool'] = cond
  df['Dif_Z_Tbreak_Bool'] = cond2
  VP = df[(df.Dif_data_Bool == True) & (df.Dif_Z_Tbreak_Bool == True )].value_counts('Dif_data_Bool').sum()
  return VP

def graphVPs(df, theta = 30, delta = (60,90), graph = True):
  """
  Para gerar o gráfico para vários finais diferentes de série é ncessário passar
  por um loop, então caso esta opção seja acessada os deltas serão definidos abaixo
  como um conjunto pré estabelecido.
  Caso se opte por não ter o gráfico só será retornado o número de VP que atende
  os parâmetros. 
  Por padrão é gerado o gráfico
  """
  df['Dif_data'] = df.End_S - df.data1_z
  df['Dif_Z_Tbreak'] = df.tBreak - df.data1_z
  #df.drop_duplicates(inplace = True)
  if graph:
    deltas = [[0,30],[30,60],[60,90],[90,120],[120,150],[150,180]]
    legenda = ['0 a 30 dias', '30 a 60 dias', '60 a 90 dias', '90 a 120 dias',
             '120 a 150 dias', '150 a 180 dias']
    qnts = []
    x=0
    for d in deltas:
      qnts.append([countVPs(df, theta, d), legenda[x]])
      x += 1
    grafDF = pd.DataFrame(qnts, columns=['VPs', 'Deltas'])
    fig, ax = plt.subplots(figsize=(13,5),dpi=80)
    matplotlib.style.use('ggplot')
    ax.scatter(grafDF.Deltas, grafDF.VPs, s = 10, c = 'y') 
    plt.ylim(grafDF.VPs.min() - 5, grafDF.VPs.max() + 5)
    for i in range(grafDF.shape[0]):
      ax.annotate(grafDF.VPs.iloc[i], xy = (grafDF.Deltas.iloc[i], grafDF.VPs.iloc[i]))                
    ax.set_title('Número de VPs X Range de datas; Theta = {0}'.format(theta))
    ax.set_ylabel('Verdadeiros Positivos')
    ax.set_xlabel('Diferença entre o Fim da Série a data Z')
    ax.plot()
    return countVPs(df, theta, delta)  
  else:
    return countVPs(df, theta, delta) 

#Exportar TIF

In [ ]:
def exportTif(params_ImgCol, points_ee, params_ccdc, output_path, export_crs, export='countBreaks', year=False): 

  """
  Creates and exports a raster with a subproduct of CCDC to the defined output path on google drive.
  Arguments:
    - params_ImgCol: parameters that define image collection;
    - points_ee: points to determine the location and extent of the study area;
    - params_ccdc: parameters used to run ccdc;
    - output_path: directory on google drive to which the raster will be saved;
    - export_crs: coordinate reference system of the output image;
    - export: defines what is the product to be exported. Options are: 
              countBreaks - raster with total number of breaks per pixel;
              largestMagBreak - raster with date in milliseconds of break of largest magnitude, stored in band 1. Associated magnitude stored in band 2;
              latestBreak - raster with date in milliseconds of the most recent break (stored in band 1) and its associated magnitude (band 2);
              specificYearBreak - raster with date in milliseconds of break in a specific year, which should be entered as a parameter. Break stored in band 1 and magnitude in band 2;
    - year: year of analysis when export is dfined to specificYearBreak.

  Function does not return anything, but saves raster to drive.
  """

  #create collection
  collection = getImageCollection(params_ImgCol, points_ee)
  #run ccdc
  ccdc_result = runCCDC(collection, params_ImgCol['bandas'], params_ccdc)
  #get tBreak and magnitude from ccdc_result
  tbreak = ccdc_result.select(['tBreak'])
  mag = ccdc_result.select(['{}_magnitude'.format(params_ImgCol['banda'])])

   
  if export == 'countBreaks':
    result = tbreak.arrayLength(0).subtract(1).toInt16()
    msg = "Generating raster with total number of breaks"
  
  elif export == 'largestMagBreak':
    if type(year) is int and year in list(range(2000,2100)):    
      """E.g ano agricola 2018 = out/2017 a set/2018, por isso year -1""" 
      # mascara para o ano agricola desejado
      anoAgric = tbreak.gt(ee.Date("{}-10-01".format(year - 1)).millis().getInfo()).And(tbreak.lt(ee.Date("{}-09-30".format(year)).millis().getInfo()))
      negativo = mag.multiply(mag.lt(0.0)).multiply(anoAgric)
      # converter para valor absoluto
      magNeg = negativo.abs()
      # converte em array com os valores maximos
      argmaxNeg = magNeg.arrayArgmax()
      # converte para pixel normal
      argmaxNegBand = argmaxNeg.arrayFlatten([['argmax']])
      # pega os valores coorrespondentes a posicao do maior argumento
      arraygetNeg = negativo.arrayGet(argmaxNegBand)
      
      # data dos tBreaks associadas a mag negativa no ano agricola selecionado
      Tnegativo = tbreak.multiply(mag.lt(0.0)).multiply(anoAgric)
      dataNeg = Tnegativo.arrayGet(argmaxNegBand)
      
      # adicionar as duas informacoes em uma unica imagem com duas bandas
      result = arraygetNeg.addBands(dataNeg)
      msg = "Generating raster with date in milliseconds of break of largest magnitude (band 1) and the associated magnitude (band 2)"
    
    else:
      raise Exception("The argument year must be passed: an integer ranging from 2000 to 2099")
    
  elif export == 'latestBreak':

    #get argmax of the array band - this will capture the most recent break
    argmax = tbreak.arrayArgmax()
    #convert array pixel to normal pixel
    argmaxband = argmax.arrayFlatten([['argmax']])
    #get elements of the pixel array according to the argmax position
    get_tbreak = tbreak.arrayGet(argmaxband)
    get_mag = mag.arrayGet(argmaxband)

    result = get_tbreak.addBands(get_mag)

    msg = "Generating raster with date in milliseconds of most recent break (band 1) and the associated magnitude of change (band 2)"

  elif export == 'specificYearBreak':

    if type(year) is int and year in list(range(2000,2100)):
      """E.g ano agricola 2018 = out/2017 a set/2018, porisso year -1"""
      #put 1 on array elements that belong to the agricultural year, otherwise put 0
      tbreak_year_flags = tbreak.gt(ee.Date("{}-10-01".format(year-1)).millis().getInfo()).And(tbreak.lt(ee.Date("{}-09-30".format(year)).millis().getInfo()))
      #get the position of 1s
      argmax = tbreak_year_flags.arrayArgmax()
      #convert array pixel to normal pixel
      argmaxband = argmax.arrayFlatten([['argmax']])
      #get elements from the original tbreak image at the obtained positions
      get_break_specific_year = tbreak.arrayGet(argmaxband)
      get_mag_specific_year = mag.arrayGet(argmaxband)

      result = get_break_specific_year.addBands(get_mag_specific_year)

      msg = "Generating raster with date in milliseconds of breaks detected within the agricultural year of {} (band 1) and the associated magnitude of change (band 2)".format(year)
    else:
      raise Exception("The argument year must be passed: an integer ranging from 2000 to 2099")

  
  else:
    raise Exception("Error: export parameter should be 'countBreaks', 'largestMagBreak', 'latestBreak' or 'specificYearBreak'.")

  #configure export parameters
  #export region
  export_region = points_ee.geometry().bounds()

  #filename - the filename needs to be set firstly to a temp_name, then it will be renamed according to the naming conventions
  #if export == 'specificYearBreak':
  if export in ['specificYearBreak', 'largestMagBreak']:
    filename = fromParamsReturnNameTiff(params_ImgCol, params_ccdc, export_region, export)+'_{}'.format(year)
  else:
    filename = fromParamsReturnNameTiff(params_ImgCol, params_ccdc, export_region, export)

  temp_name = 'temp_{}'.format(random.randint(0,9999))

  #set folder to which GEE will export the image
  folder = os.path.split(output_path)[-1]

  #export raster to drive
  task = ee.batch.Export.image.toDrive(**{
      'image':result,
      'description': temp_name, #filename,
      'folder':folder,
      'scale':10,
      'region': export_region,
      'crs': export_crs,
      'maxPixels':8030040147504
  })

  task.start()
  t_start = time.time()
  print(msg)
  print('\n')
  while task.active():
    print('\rRunning export task... Time elapsed: {} min'.format(str(round((time.time()-t_start)/60,2))), end='')
    time.sleep(5)

  #rename
  old_name = os.path.join(output_path, temp_name+'.tif')
  new_name = os.path.join(output_path, filename+'.tif')
  #check if file was already uploaded to google drive and then rename it
  found = False
  while not found:
    if os.path.exists(old_name):
      found = True
    time.sleep(1)
  os.rename(old_name, new_name)
  print('\nExport finished')



#Funcoes auxiliares

#####loadValidationPoints

In [ ]:
def loadValidationPoints(points, crs='epsg:4326', id=False):

  """
  Returns the validation points as a geopandas dataframe and
  as an ee feature collection (or an ee feature in the case of 1 only point).
  """

  #execute if a path was entered
  if type(points) is str:
    if points.endswith('.shp') or points.endswith('.gpkg'):
      gdf_points = gpd.read_file(points)
    elif points.endswith('.csv'):
      gdf_points = gpd.GeoDataFrame(pd.read_csv(points), crs=crs)
    elif points.endswith('.pkl'):
      gdf_points = pd.read_pickle(points)
    #check if there is an id field in the gdf, then rename it to OBID. If there isn't, create an OBID field
    if 'OBID' not in gdf_points.columns:
      if 'id' in gdf_points.columns:
        gdf_points = gdf_points.rename(columns={"id": "OBID"})
      else:
        gdf_points['OBID'] = gdf_points.index
    #check if a specific id was entered
    if id is not False:
      gdf_points = gdf_points[gdf_points['OBID']==id]
    #convert gpd to feature collection
    ee_fc_points = geemap.geopandas_to_ee(gdf_points)

  #execute if list of coordinates was entered
  elif type(points) is list:

    if len(points)==1: #enter here if only one point was provided by the user
      #create gdf_points
      #create dataframe with Long, Lat and OBID
      temp_df = pd.DataFrame({'Long':[points[0][0]], 'Lat':[points[0][1]], 'OBID':0})
      #create geodataframe from dataframe 
      gdf_points = gpd.GeoDataFrame(temp_df, geometry=gpd.points_from_xy(temp_df.Long, temp_df.Lat))
      #drop Long and Lat columns, then assign WGS84 CRS
      gdf_points.drop(columns=['Long','Lat'], inplace=True)
      gdf_points.crs = 'EPSG:4326'

      #create ee_fc_points - note that in this case it is an ee.feature instead of a feature collection
      ee_fc_points = getSpecificPoint(points[0][0], points[0][1], 0)
    
    else: #in the case more than 1 points were entered by the user
      list_eeFeatures = []
      #adds ee Features (Points) to list
      for i in range(len(points)):
        list_eeFeatures.append(getSpecificPoint(points[i][0], points[i][1], i))
      #create feature collection
      ee_fc_points = ee.FeatureCollection(list_eeFeatures)
      #convert the feature collection to gdf
      gdf_points = geemap.ee_to_geopandas(ee_fc_points)

  else:
    raise Exception("loadValidationPoints function argument should be a string with a path or a list of coordinates.") 

  return gdf_points, ee_fc_points

#####getSpecificPoint

In [ ]:
def getSpecificPoint(long_ponto, lat_ponto, id):

  """
  Returns an ee.feature (Point) based on the coordinates entered. Also sets the id attribute.
  """

  ponto_especifico_ee = ee.Feature(ee.Geometry.Point([long_ponto,lat_ponto]), {'OBID':id})

  return ponto_especifico_ee

#####fromParamsReturnName

In [ ]:
def fromParamsReturnName(params_ImgCol, params_ccdc, Plon, Plat, buffer):

  """Returns the filename according to the parameters of execution."""

  #returns the name of the file (without extension) based on the parameters entered.

  #get collection name
  col_name = "S2SR" if params_ImgCol['nameImage'] == "COPERNICUS/S2_SR" else params_ImgCol['nameImage']
  
  #get indices
  aux = ''
  params_ImgCol['indices'].sort()
  for i in params_ImgCol['indices']:
    indices = aux + i
    aux = indices + '-'
  indices = indices.upper()

  #get ccdc params
  #get tmask bands
  tmask = ''
  for i in params_ccdc['bandas_tmask']:
    tmask = tmask + i
  tmask = tmask.upper()
  #get chi
  chi = str(params_ccdc['chiSquare'])
  chi = chi[chi.find('.')+1:]
  #get minYears
  minYears = str(params_ccdc['minYears']).replace('.','')
  #get num obs
  n_obs = str(params_ccdc['minObs'])
  #get lambda
  lam = str(params_ccdc['Lambda'])
  #get max iter
  maxIter = str(params_ccdc['maxIter'])

  #get point lat/lon -> format: IDDDDD, IIDDDDD or IIIDDDDD followed by N, S, E or W (N, E for positive, S, W for negative)
  lat = "{:7.5f}N".format(Plat).replace('.','') if Plat > 0 else "{:7.5f}S".format(abs(Plat)).replace('.','')
  lon = "{:7.5f}E".format(Plon).replace('.','') if Plon > 0 else "{:7.5f}W".format(abs(Plon)).replace('.','')

  #get buffer size
  buf = 0 if buffer is False else str(buffer) 

  #get dates
  date_start = params_ImgCol['date_start'].replace('-','')
  date_end = params_ImgCol['date_end'].replace('-','')

  #get cloudmask
  cmask = 'S2CLESS' if params_ImgCol['cloudFilter'] == 's2cloudless' else params_ImgCol['cloudFilter'].upper()
  

  name = "{0}-{1}_{2}_TMASK{3}XX{4}YM{5}NOBS{6}LDA{7}ITER{8}_LON{9}_LAT{10}_BUF{11}_START{12}_END{13}".format(col_name, cmask, indices, tmask, chi, minYears, n_obs, lam, maxIter,
                                                                                                        lon, lat, buf, date_start, date_end)
  
  return name

#####fromParamsReturnNameTiff

In [ ]:
def fromParamsReturnNameTiff(params_ImgCol, params_ccdc, export_region, var):

  """Returns the filename according to the parameters of execution.
  export_region must be a geometry"""

  #returns the name of the file (without extension) based on the parameters entered.

  #get collection name
  col_name = "S2SR" if params_ImgCol['nameImage'] == "COPERNICUS/S2_SR" else params_ImgCol['nameImage']
  
  #get indices
  aux = ''
  params_ImgCol['indices'].sort()
  for i in params_ImgCol['indices']:
    indices = aux + i
    aux = indices + '-'
  indices = indices.upper()

  #get ccdc params
  #get tmask bands
  tmask = ''
  for i in params_ccdc['bandas_tmask']:
    tmask = tmask + i
  tmask = tmask.upper()
  #get chi
  chi = str(params_ccdc['chiSquare'])
  chi = chi[chi.find('.')+1:]
  #get minYears
  minYears = str(params_ccdc['minYears']).replace('.','')
  #get num obs
  n_obs = str(params_ccdc['minObs'])
  #get lambda
  lam = str(params_ccdc['Lambda'])
  #get max iter
  maxIter = str(params_ccdc['maxIter'])

  #get point lat/lon -> format: IDDDDD, IIDDDDD or IIIDDDDD followed by N,S,E or W (N,E for positive, S,W for negative)
  bounds = export_region #export_region.geometry().bounds()
  listCoords = ee.Array.cat(bounds.coordinates(), 1)
  xCoords = listCoords.slice(1, 0, 1)
  yCoords = listCoords.slice(1, 1, 2)
  lonmin = xCoords.reduce('min', [0]).get([0,0]).getInfo()
  lonmax = xCoords.reduce('max', [0]).get([0,0]).getInfo()
  latmin = yCoords.reduce('min', [0]).get([0,0]).getInfo()
  latmax = yCoords.reduce('max', [0]).get([0,0]).getInfo()

  lonmin = "{:7.5f}E".format(lonmin).replace('.','') if lonmin > 0 else "{:7.5f}W".format(abs(lonmin)).replace('.','')
  lonmax = "{:7.5f}E".format(lonmax).replace('.','') if lonmax > 0 else "{:7.5f}W".format(abs(lonmax)).replace('.','')
  latmin = "{:7.5f}N".format(latmin).replace('.','') if latmin > 0 else "{:7.5f}S".format(abs(latmin)).replace('.','')
  latmax = "{:7.5f}N".format(latmax).replace('.','') if latmax > 0 else "{:7.5f}S".format(abs(latmax)).replace('.','')

  #get dates
  date_start = params_ImgCol['date_start'].replace('-','')
  date_end = params_ImgCol['date_end'].replace('-','')



  name = "{0}_{1}_{2}_TMASK{3}XX{4}YM{5}NOBS{6}LDA{7}ITER{8}_LONMIN{9}_LONMAX{10}_LATMIN{11}_LATMAX{12}_START{13}_END{14}_VAR{15}".format(
                                                                                      col_name, params_ImgCol['banda'], indices, tmask, chi, minYears, n_obs, lam, maxIter,
                                                                                                        lonmin, lonmax, latmin, latmax, date_start, date_end, var)
  
  return name

#####deleteItems

In [ ]:
def deleteItems(path, arg):
  """
  Deletar varios itens do drive
  - entrada:
  path: pasta onde os arquivos se encontram
  arg: argumentos que devem constar no nome,
       e.g '*210*.csv', encontra todos os arquivos que tenham 210 no nome e terminem com .csv
  - saida: não há
  """
  os.chdir(path) # transforma a pasta de entrada no atual workspace
  listaArquivo = glob.glob(arg) # procura na pasta de workspace todos os arquivos que contenham o argumento dado
  for arquivo in listaArquivo:
    print('Removido:', arquivo)
    os.remove(arquivo)

#Compositos

####createComposite

In [ ]:
def fillImgColGapsWithCCDC(params_ImgCol, bands, params_ccdc, points_ee, buf=False):

  """
  Function that fills gaps in an image collection based on CCDC fitted segments. 
  Returns a gap free image collection.
  Arguments:
  params_ImgCol - parameters necessary to initilize image collection;
  bands - bands that should be added to the composite;
  params_ccdc - parameters used to run CCDC;
  points_ee - ee feature containing the points of interest;
  buf - whether to process the information for a buffer around the points.
  """
  #transform bands into an ee.List object
  bandList = ee.List(bands)

  #initialize image collection
  imgCollection = getImageCollection(params_ImgCol, points_ee, buf=buf)

  #select bands of interest
  imgCollection = imgCollection.select(bands)

  #run ccdc
  ccdc = runCCDC(imgCollection, bands, params_ccdc)

  #add time band to images
  def addTime(img):
    time = ee.Image(img.date().millis()).rename('t').float();
    return img.addBands(time)
  
  imgCollection =  imgCollection.map(addTime)

  harmonicFrequencies = ee.List.sequence(1, 3);

  def getCoefs(img):
    #get position that represents the segment
    segment = img.select('t').gt(ccdc.select('tEnd')).Not().arrayArgmax().arrayFlatten([['tEnd']])
    t_end_check = img.select('t').gt(ccdc.select('tEnd').arrayGet(-1))
    segment = segment.where(t_end_check, ccdc.select('tEnd').arrayLength(0).add(-1))

    #make an image of frequencies
    frequencies = ee.Image.constant(harmonicFrequencies);
    #calculate time in randians (considering time is originally in milliseconds)
    PI2 = 2.0 * np.pi
    time = ee.Image(img).select('t').multiply( PI2 / (1000 * 60 * 60 * 24 * 365.25))
    #get the cosine terms
    cosines = time.multiply(frequencies).cos().rename(['cos1','cos2','cos3'])
    #get the sin terms
    sines = time.multiply(frequencies).sin().rename(['sin1','sin2','sin3'])
    #compute terms array
    terms = ee.Image.cat(ee.Image.constant(1),img.select('t'),cosines.select('cos1'),sines.select('sin1'),
                                                          cosines.select('cos2'),sines.select('sin2'),
                                                          cosines.select('cos3'),sines.select('sin3')).toArray().rename('terms')
    #get coefficients per band, multiply by their correspondent terms (sin, cos, slope, interception) and compute sum
    def getCoefsPerBand(b):
      return ccdc.select(ee.String(b).cat('_coefs')).arraySlice(0,segment,segment.add(1)).arrayReshape(ee.Image.constant(ee.Array([8])),1).multiply(terms).arrayReduce(ee.Reducer.sum(),[0]).arrayGet(0).rename(ee.String(b).cat('_fitted'))                                                    
    bands_coef = bandList.map(getCoefsPerBand)
      
    def addBandsIter(b,previous):
      return ee.Image(previous).addBands(b)
    merged = bands_coef.flatten().iterate(addBandsIter, ee.Image())
    
    return img.addBands(merged)

  fittedCol = imgCollection.map(getCoefs)
  
  def fillGaps(img):
    def addBandsIter2(b,previous):
      return ee.Image(previous).addBands(img.select(ee.String(b)).unmask(img.select(ee.String(b).cat('_fitted'))))
    temp = img.select('t').addBands(bandList.iterate(addBandsIter2, ee.Image()))
                  
    return temp.select(bandList)

  #mask
  mask_ccdc = ccdc.select('tStart').mask()
  def maskMasked(img):
    return img.updateMask(mask_ccdc)
  fittedCol = fittedCol.map(maskMasked)

  return fittedCol.map(fillGaps)


In [ ]:
def createComposite(imgCollection, composite_period):

  """
  Function that creates monthly composites (median) of an image collection, returns an ee.Image.
  Arguments: 
  imgCollection - image collection (time series) used to create composites;
  composite_period - a dictionary that contains the dates of start and end, determining the time period
                  that the composite will cover.
  """

  #get start and end years
  year_start = ee.Date(composite_period['date_start']).get('year')
  year_end = ee.Date(composite_period['date_end']).get('year')

  #create list with 12 months and the intended years
  months = ee.List.sequence(1,12)
  years = ee.List.sequence(year_start,year_end)

  ############### auxiliary functions ###############
  #block 1 - create composites and rename images of the composite
  #auxiliary functions to run the createComposite function (they have to be defined within the function to allow the use of the map operator)
  def makeCompositesPerYear(y):

    def makeMonthlyComposites(m):
      return imgCollection.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m,m,'month')).median().int().set('month',m,'year',y)

    return ee.ImageCollection.fromImages(months.map(makeMonthlyComposites))
  
  def decomposeList(l):
    return ee.ImageCollection(l).toList(12)

  def renameImages(img):
    eeimg = ee.Image(img)
    value = ee.Number(eeimg.get('year')).format('%04d').cat('_').cat(ee.Number(eeimg.get('month')).format('%02d'))
    return ee.Image(eeimg.set('system:index',value,'system:id',value))

  #block 2 - set and add image time
  #auxiliary function to set the time of the image, set number of bands and add a time band
  def setTimeAndNumBands(img):
    return img.set('system:time_start', ee.Date.fromYMD(img.get('year'),img.get('month'),1).millis(), 'num_bands', img.bandNames().length())
  
  ############### end of auxiliary functions ###############

  #execution of block 1 - creation of the composite
  #make composites per year
  list_years = years.map(makeCompositesPerYear)

  #decompose resulting img collection to list of images
  list_imgs = list_years.map(decomposeList).flatten()

  #rename images (format name as YYYY_MM)
  list_imgs_renamed = list_imgs.map(renameImages)

  #convert from list to img collection
  decomposed_collection = ee.ImageCollection(list_imgs_renamed)

  #transform images of the image collection in bands of single image 
  #composite_in_bands = decomposed_collection.toBands()

  #execution of block 2 - add variables
  #decomposed_collection = decomposed_collection.map(addVariables)
  
  decomposed_collection = decomposed_collection.map(setTimeAndNumBands)
  decomposed_collection = decomposed_collection.filter('num_bands > 0')

  return decomposed_collection.toBands()#.select(band_names)



#### Extrair info dos compósitos

In [ ]:
#OBS - FUNCAO NÃO APROPRIADA PARA GERAR O DATAFRAME TOTAL, POIS NECESSITA DE MUITO TEMPO DE EXECUÇÃO
#ESTÁ AQUI APENAS COMO MATERIAL DE BASE PARA FUTUROS AJUSTES/ADAPTAÇÕES, SE NECESSÁRIO
#O PROCEDIMENTO UTILIZADO PARA GERAR OS COMPÓSITOS SALVOS NO DRIVE É DESCRITO NO MAIN
def extractCompositeInfo(df, params_ImgCol, composite_period, bands_composite, params_ccdc, points_ee, buf=False):

  #compute gap free image collection
  filledCol = fillImgColGapsWithCCDC(params_ImgCol, bands_composite, params_ccdc, points_ee, buf=False)

  #generate composite
  composite = createComposite(filledCol, composite_period)

  #initialize empty dataframe
  df_composites = pd.DataFrame()

  #adjust coordinate column of the dataframe - if it is str, convert to tuple
  #if type(df.coord_ccdc.head(1)[0]) is str:
    #df.coord_ccdc = df.coord_ccdc.apply(eval)
  
  #transform to WGS84 if in other crs
  if df.crs.name != 'WGS 84':
    df = df.to_crs('epsg:4326')

  #loop through dataframe rows and get lat lon
  #for Plat, Plon in list(df.coord_ccdc):
  for Plat, Plon in list(zip(df.geometry.y.values, df.geometry.x.values)):
    print(Plon, Plat)
    
    if buf:
      target_geometry = ee.Feature(ee.Geometry.Point([Plon, Plat])).buffer(buf).geometry()
    else:
      target_geometry = ee.Geometry.Point([Plon, Plat]) 

    #unmask - fill masked elements with an arbitrary value
    composite = composite.unmask(32767)

    #apply reduceRegion
    reduced = composite.reduceRegion(
        reducer= ee.Reducer.toList(),    
        geometry = target_geometry,#ee.Geometry.Point([Plon,Plat]),    
        scale= 10)  
    
    #store in temporary dataframe
    df_temp = pd.DataFrame(reduced.getInfo()) 
    print('chegou aqui')
    #add coordinates
    dicLatLong = ee.Image.pixelLonLat().updateMask(composite.mask().reduce('anyNonZero')).reduceRegion(
      reducer= ee.Reducer.toList(),
      geometry = target_geometry,
      scale= 10)
    #convert to data frame
    df_Lat_Long = pd.DataFrame(dicLatLong.getInfo())
    #join
    df_temp = df_temp.join(df_Lat_Long)

    #append temporary dataframe to initial dataframe
    df_composites = df_composites.append(df_temp)
    print('completed')
 
    
  return df_composites